In [1]:
import os, json, re
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterstats import zonal_stats
import osmnx as ox
import statsmodels.api as sm
from shapely.geometry import box, Point
from shapely.geometry import mapping
from IPython.display import IFrame

# -----------------------------
# Project / output folders
# -----------------------------
DATA_DIR = "data/yerevan_full_pipeline"
os.makedirs(DATA_DIR, exist_ok=True)

# -----------------------------
# Grid settings
# -----------------------------
GRID_CELL_M = 500          # 250 gives more detail, larger HTML
BUFFER_HERIT_M = 500       # heritage count radius

# -----------------------------
# Transport proxy settings
# -----------------------------
AVG_SPEED_KMH = 20.0
SPEED_M_PER_MIN = AVG_SPEED_KMH * 1000.0 / 60.0

# -----------------------------
# Rent conversion settings
# -----------------------------
GROSS_YIELD_ANNUAL = 0.06  # implied rent from sale price per m2
FILTER_CURRENCY = "USD"    # keep USD only for Kaggle sale listing

# -----------------------------
# Inputs
# -----------------------------
WORLDPOP_URL = (
    "https://data.worldpop.org/GIS/Population/Individual_countries/ARM/"
    "Armenia_100m_Population/ARM_ppp_v2c_2020.tif"
)

KAGGLE_SALE_CSV = r"data/kaggle_real_estate/apartments_for_sale(with_lat_long).csv"

# Historic center g (Republic Square) fixed coordinates
G_LON, G_LAT = 44.5126, 40.1775  # lon, lat


In [2]:
def ensure_crs(gdf, epsg=32638):
    if gdf.crs is None:
        raise RuntimeError("GeoDataFrame CRS missing.")
    return gdf.to_crs(epsg=epsg)

def download_file(url, out_path):
    import requests
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    if os.path.exists(out_path) and os.path.getsize(out_path) > 0:
        return out_path
    r = requests.get(url, stream=True, timeout=180)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk)
    return out_path

def get_yerevan_boundary():
    ox.settings.use_cache = True
    ox.settings.log_console = True
    ox.settings.timeout = 180

    gdf = ox.geocode_to_gdf("Yerevan, Armenia")
    gdf = gdf[gdf.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()
    if gdf.empty:
        raise RuntimeError("Could not fetch Yerevan polygon from OSM.")
    try:
        geom = gdf.geometry.union_all()
    except Exception:
        geom = gdf.geometry.unary_union
    out = gpd.GeoDataFrame({"name":["Yerevan"]}, geometry=[geom], crs=gdf.crs)
    return out

def clip_raster_to_polygon(in_tif, poly_gdf, out_tif):
    with rasterio.open(in_tif) as src:
        poly = poly_gdf.to_crs(src.crs)
        geom = poly.geometry.iloc[0]
        geoms = [mapping(geom)]
        out_img, out_transform = mask(src, geoms, crop=True, filled=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "height": out_img.shape[1],
            "width": out_img.shape[2],
            "transform": out_transform
        })
        os.makedirs(os.path.dirname(out_tif), exist_ok=True)
        with rasterio.open(out_tif, "w", **out_meta) as dst:
            dst.write(out_img)
    return out_tif

def reproject_raster(in_tif, out_tif, dst_crs="EPSG:32638"):
    os.makedirs(os.path.dirname(out_tif), exist_ok=True)
    with rasterio.open(in_tif) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({"crs": dst_crs, "transform": transform, "width": width, "height": height})
        with rasterio.open(out_tif, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )
    return out_tif

def make_grid(bounds, cell_size_m, crs):
    minx, miny, maxx, maxy = bounds
    xs = np.arange(minx, maxx + cell_size_m, cell_size_m)
    ys = np.arange(miny, maxy + cell_size_m, cell_size_m)
    cells = []
    for x in xs[:-1]:
        for y in ys[:-1]:
            cells.append(box(x, y, x + cell_size_m, y + cell_size_m))
    grid = gpd.GeoDataFrame({"geometry": cells}, crs=crs)
    grid["cell_id"] = np.arange(len(grid)).astype(int)
    return grid

def parse_area(x):
    if pd.isna(x):
        return np.nan
    s = str(x).replace(",", ".")
    m = re.search(r"(\d+(\.\d+)?)", s)
    return float(m.group(1)) if m else np.nan

def utm_xy_to_lonlat(x, y):
    pt = gpd.GeoSeries([Point(x, y)], crs="EPSG:32638").to_crs("EPSG:4326").iloc[0]
    return float(pt.x), float(pt.y)


In [3]:
yerevan_ll = get_yerevan_boundary()               # EPSG:4326 usually
yerevan_utm = ensure_crs(yerevan_ll, 32638)

BOUNDARY_GPKG = os.path.join(DATA_DIR, "yerevan_boundary.gpkg")
yerevan_utm.to_file(BOUNDARY_GPKG, layer="boundary", driver="GPKG")

print("Saved boundary:", BOUNDARY_GPKG)


Saved boundary: data/yerevan_full_pipeline\yerevan_boundary.gpkg


In [4]:
raw_tif = os.path.join(DATA_DIR, "worldpop_ARM_2020.tif")
clip_tif = os.path.join(DATA_DIR, "worldpop_yerevan_clipped_wgs84.tif")
clip_utm_tif = os.path.join(DATA_DIR, "worldpop_yerevan_clipped_utm38n.tif")

download_file(WORLDPOP_URL, raw_tif)
clip_raster_to_polygon(raw_tif, yerevan_ll, clip_tif)
reproject_raster(clip_tif, clip_utm_tif, dst_crs="EPSG:32638")

print("Saved raster:", clip_utm_tif)


Saved raster: data/yerevan_full_pipeline\worldpop_yerevan_clipped_utm38n.tif


In [5]:
grid = make_grid(yerevan_utm.total_bounds, GRID_CELL_M, crs=yerevan_utm.crs)
grid = gpd.overlay(grid, yerevan_utm[["geometry"]], how="intersection")
grid["area_km2"] = grid.geometry.area / 1_000_000.0
grid["cx"] = grid.geometry.centroid.x
grid["cy"] = grid.geometry.centroid.y

GRID_GPKG = os.path.join(DATA_DIR, f"yerevan_grid_{GRID_CELL_M}m.gpkg")
grid.to_file(GRID_GPKG, layer="grid", driver="GPKG")

print("Saved grid:", GRID_GPKG, "cells:", len(grid))


Saved grid: data/yerevan_full_pipeline\yerevan_grid_500m.gpkg cells: 1036


In [6]:
grid = make_grid(yerevan_utm.total_bounds, GRID_CELL_M, crs=yerevan_utm.crs)
grid = gpd.overlay(grid, yerevan_utm[["geometry"]], how="intersection")
grid["area_km2"] = grid.geometry.area / 1_000_000.0
grid["cx"] = grid.geometry.centroid.x
grid["cy"] = grid.geometry.centroid.y

GRID_GPKG = os.path.join(DATA_DIR, f"yerevan_grid_{GRID_CELL_M}m.gpkg")
grid.to_file(GRID_GPKG, layer="grid", driver="GPKG")

print("Saved grid:", GRID_GPKG, "cells:", len(grid))


Saved grid: data/yerevan_full_pipeline\yerevan_grid_500m.gpkg cells: 1036


In [7]:
with rasterio.open(clip_utm_tif) as src:
    nodata = src.nodata

zs = zonal_stats(grid, clip_utm_tif, stats=["sum"], nodata=nodata, all_touched=False)
grid_pop = grid.copy()
grid_pop["pop_sum"] = [z["sum"] if z["sum"] is not None else 0.0 for z in zs]
grid_pop["pop_density_per_km2"] = grid_pop["pop_sum"] / grid_pop["area_km2"].replace(0, np.nan)

POP_GPKG = os.path.join(DATA_DIR, "grid_population.gpkg")
grid_pop.to_file(POP_GPKG, layer="grid_pop", driver="GPKG")
print("Saved population grid:", POP_GPKG)


Saved population grid: data/yerevan_full_pipeline\grid_population.gpkg


In [8]:
ox.settings.use_cache = True
ox.settings.log_console = True
ox.settings.timeout = 180

BUSINESS_TAGS = {
    "shop": True,
    "office": True,
    "craft": True,
    "industrial": True,
    "amenity": ["restaurant","cafe","bar","fast_food","bank","pharmacy","hospital","clinic"],
    "building": ["commercial","industrial","retail","office"]
}

poly_ll = yerevan_ll.geometry.iloc[0]
biz_raw = ox.features_from_polygon(poly_ll, BUSINESS_TAGS)
biz_raw = biz_raw[biz_raw.geometry.notnull()].copy()

biz = ensure_crs(biz_raw, 32638)
gt = biz.geometry.geom_type
pts = biz[gt == "Point"].copy()
polys = biz[gt.isin(["Polygon","MultiPolygon"])].copy()
polys["geometry"] = polys.geometry.representative_point()
biz_pts = gpd.GeoDataFrame(pd.concat([pts, polys], ignore_index=True), crs="EPSG:32638")

# Spatial join to grid
join = gpd.sjoin(biz_pts[["geometry"]], grid[["cell_id","geometry"]], predicate="within", how="left")
counts = join.groupby("cell_id").size().rename("biz_count").reset_index()

grid_biz = grid.copy()
grid_biz = grid_biz.merge(counts, on="cell_id", how="left")
grid_biz["biz_count"] = grid_biz["biz_count"].fillna(0).astype(int)
grid_biz["biz_density_per_km2"] = grid_biz["biz_count"] / grid_biz["area_km2"].replace(0, np.nan)

# Keep only safe columns for saving points (you don't need all OSM attributes)
safe_cols = [c for c in ["name", "shop", "office", "craft", "industrial", "amenity", "building"] if c in biz_pts.columns]
biz_pts_out = biz_pts[safe_cols + ["geometry"]].copy()

BIZ_POINTS_GPKG = os.path.join(DATA_DIR, "business_points.gpkg")
BIZ_GRID_GPKG = os.path.join(DATA_DIR, "grid_business.gpkg")

biz_pts_out.to_file(BIZ_POINTS_GPKG, layer="biz_points", driver="GPKG")
grid_biz.to_file(BIZ_GRID_GPKG, layer="grid_biz", driver="GPKG")

print("Saved business points:", BIZ_POINTS_GPKG, "n:", len(biz_pts))
print("Saved business grid:", BIZ_GRID_GPKG)


Saved business points: data/yerevan_full_pipeline\business_points.gpkg n: 10817
Saved business grid: data/yerevan_full_pipeline\grid_business.gpkg


In [9]:
df = pd.read_csv(KAGGLE_SALE_CSV)
for c in ["Latitude","Longitude","price","currency","floor_area"]:
    if c not in df.columns:
        raise RuntimeError(f"Missing column {c} in Kaggle CSV. Columns: {list(df.columns)}")

work = df.dropna(subset=["Latitude","Longitude","price"]).copy()
work["price"] = pd.to_numeric(work["price"], errors="coerce")
work = work.dropna(subset=["price"])

work["currency"] = work["currency"].astype(str).str.upper().str.strip()
work = work[work["currency"] == FILTER_CURRENCY].copy()

work["area_m2"] = work["floor_area"].apply(parse_area)
work.loc[(work["area_m2"] < 10) | (work["area_m2"] > 500), "area_m2"] = np.nan
work["price_per_m2"] = work["price"] / work["area_m2"]
work.loc[(work["price_per_m2"] <= 0) | (work["price_per_m2"] > 20000), "price_per_m2"] = np.nan

# Implied rent per m2 per month
work["implied_rent_per_m2_month"] = (work["price_per_m2"] * GROSS_YIELD_ANNUAL) / 12.0

rent_pts = gpd.GeoDataFrame(
    work,
    geometry=gpd.points_from_xy(work["Longitude"], work["Latitude"]),
    crs="EPSG:4326"
)
# clip to Yerevan
rent_pts = rent_pts[rent_pts.within(yerevan_ll.geometry.iloc[0])].copy()
rent_pts = ensure_crs(rent_pts, 32638)

# Join to grid and aggregate
j = gpd.sjoin(rent_pts[["implied_rent_per_m2_month","geometry"]],
              grid[["cell_id","geometry"]],
              predicate="within", how="left")

agg = j.groupby("cell_id").agg(
    rent_listings=("geometry","size"),
    implied_rent_per_m2_month=("implied_rent_per_m2_month","median")
).reset_index()

grid_rent = grid.copy()
grid_rent = grid_rent.merge(agg, on="cell_id", how="left")
grid_rent["rent_listings"] = grid_rent["rent_listings"].fillna(0).astype(int)

RENT_POINTS_GPKG = os.path.join(DATA_DIR, "rent_points_kaggle_sale.gpkg")
RENT_GRID_GPKG = os.path.join(DATA_DIR, "grid_rent.gpkg")
rent_pts.to_file(RENT_POINTS_GPKG, layer="rent_points", driver="GPKG")
grid_rent.to_file(RENT_GRID_GPKG, layer="grid_rent", driver="GPKG")

print("Saved rent points:", RENT_POINTS_GPKG, "n:", len(rent_pts))
print("Saved rent grid:", RENT_GRID_GPKG)


Saved rent points: data/yerevan_full_pipeline\rent_points_kaggle_sale.gpkg n: 56021
Saved rent grid: data/yerevan_full_pipeline\grid_rent.gpkg


In [10]:
TAGS_HERIT = {
    "historic": True,
    "heritage": True,
    "tourism": ["museum", "attraction"],
    "memorial": True
}
herit_raw = ox.features_from_polygon(yerevan_ll.geometry.iloc[0], TAGS_HERIT)
herit_raw = herit_raw[herit_raw.geometry.notnull()].copy()
herit = ensure_crs(herit_raw, 32638)

gt = herit.geometry.geom_type
hp = herit[gt == "Point"].copy()
hpoly = herit[gt.isin(["Polygon","MultiPolygon"])].copy()
hpoly["geometry"] = hpoly.geometry.representative_point()
herit_pts = gpd.GeoDataFrame(pd.concat([hp, hpoly], ignore_index=True), crs="EPSG:32638")

# g point in UTM
g_utm = gpd.GeoSeries([Point(G_LON, G_LAT)], crs="EPSG:4326").to_crs("EPSG:32638").iloc[0]

amen = grid.copy()
amen_cent = gpd.GeoDataFrame(amen[["cell_id"]].copy(), geometry=amen.geometry.centroid, crs="EPSG:32638")

# distance to g
amen["dist_to_g_m"] = amen.geometry.centroid.distance(g_utm)

# distance to nearest heritage point
nearest = gpd.sjoin_nearest(
    amen_cent,
    herit_pts[["geometry"]],
    how="left",
    distance_col="dist_to_nearest_herit_m"
)
amen = amen.merge(nearest[["cell_id","dist_to_nearest_herit_m"]], on="cell_id", how="left")

# count within buffer
buff = amen_cent.copy()
buff["geometry"] = buff.geometry.buffer(BUFFER_HERIT_M)
join_cnt = gpd.sjoin(herit_pts[["geometry"]], buff[["cell_id","geometry"]], predicate="within", how="left")
cnt = join_cnt.groupby("cell_id").size().rename("herit_cnt_500m").reset_index()
amen = amen.merge(cnt, on="cell_id", how="left")
amen["herit_cnt_500m"] = amen["herit_cnt_500m"].fillna(0).astype(int)

buffer_area_km2 = (np.pi * (BUFFER_HERIT_M ** 2)) / 1_000_000.0
amen["herit_density_500m"] = amen["herit_cnt_500m"] / buffer_area_km2

AMEN_POINTS_GPKG = os.path.join(DATA_DIR, "heritage_points.gpkg")
AMEN_GRID_GPKG = os.path.join(DATA_DIR, "grid_amenity.gpkg")
herit_pts.to_file(AMEN_POINTS_GPKG, layer="herit_points", driver="GPKG")
amen.to_file(AMEN_GRID_GPKG, layer="grid_amen", driver="GPKG")

print("Saved heritage points:", AMEN_POINTS_GPKG, "n:", len(herit_pts))
print("Saved amenity grid:", AMEN_GRID_GPKG)


Saved heritage points: data/yerevan_full_pipeline\heritage_points.gpkg n: 741
Saved amenity grid: data/yerevan_full_pipeline\grid_amenity.gpkg


In [11]:
import os, json
import pandas as pd
import geopandas as gpd
import osmnx as ox
import folium
from folium.plugins import MarkerCluster
from shapely.geometry import Point
from IPython.display import IFrame

# ------------------------------------------------------------
# Helpers
# ------------------------------------------------------------
def _jsonify_weird_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Convert list/dict/set/tuple columns to JSON strings so CSV/GPKG writing is safe."""
    out = df.copy()
    for c in out.columns:
        if c == "geometry":
            continue
        if out[c].apply(lambda x: isinstance(x, (list, dict, set, tuple))).any():
            out[c] = out[c].apply(
                lambda x: json.dumps(x, ensure_ascii=False)
                if isinstance(x, (list, dict, set, tuple))
                else x
            )
    return out

def _safe_str(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return ""
    return str(x)

# ------------------------------------------------------------
# 1) OSM tags: heritage + monuments (and related)
# ------------------------------------------------------------
# Your original tags
TAGS_HERIT_BASE = {
    "historic": True,
    "heritage": True,
    "tourism": ["museum", "attraction"],
    "memorial": True
}

# Added: monuments and common monument-like tagging patterns
TAGS_MONUMENTS_1 = {"historic": ["monument", "memorial"]}
TAGS_MONUMENTS_2 = {"tourism": ["artwork", "gallery", "attraction"]}
TAGS_MONUMENTS_3 = {"memorial": ["statue", "bust", "stone", "plaque", "sculpture", "war_memorial"]}
# Some mappers also use these keys directly
TAGS_MONUMENTS_4 = {"artwork_type": True}

tag_queries = [TAGS_HERIT_BASE, TAGS_MONUMENTS_1, TAGS_MONUMENTS_2, TAGS_MONUMENTS_3, TAGS_MONUMENTS_4]

# Make sure yerevan_ll polygon is in EPSG:4326 for OSMnx querying
poly_ll = yerevan_ll.to_crs("EPSG:4326").geometry.iloc[0]

frames = []
for tags in tag_queries:
    try:
        g = ox.features_from_polygon(poly_ll, tags)
        if g is not None and len(g) > 0:
            frames.append(g)
    except Exception as e:
        print("OSMnx query failed for tags:", tags, "| error:", repr(e))

if len(frames) == 0:
    raise RuntimeError("No OSM features were returned. Check polygon, internet, or Overpass availability.")

# Combine and deduplicate by index (OSMnx typically uses a multiindex like (element_type, osmid))
herit_raw = pd.concat(frames)
herit_raw = herit_raw[~herit_raw.index.duplicated(keep="first")].copy()
herit_raw = herit_raw[herit_raw.geometry.notnull()].copy()

# Project to UTM (your pipeline uses EPSG:32638)
herit = ensure_crs(herit_raw, 32638)

# ------------------------------------------------------------
# 2) Convert to datapoints (keep original geometry as WKT too)
# ------------------------------------------------------------
herit = herit.copy()
herit["orig_geom_type"] = herit.geometry.geom_type
herit["orig_geom_wkt"] = herit.geometry.to_wkt()

# Convert non-point geometries to representative points
points = herit[herit["orig_geom_type"] == "Point"].copy()
non_points = herit[herit["orig_geom_type"] != "Point"].copy()

if len(non_points) > 0:
    non_points = non_points.copy()
    non_points["geometry"] = non_points.geometry.representative_point()

herit_pts = gpd.GeoDataFrame(
    pd.concat([points, non_points], ignore_index=True),
    crs="EPSG:32638",
    geometry="geometry"
)

# Add a stable local id (useful for joins, popups, etc.)
herit_pts["feature_id"] = range(1, len(herit_pts) + 1)

# g point in UTM for distance calc (you already had this)
g_utm = gpd.GeoSeries([Point(G_LON, G_LAT)], crs="EPSG:4326").to_crs("EPSG:32638").iloc[0]
herit_pts["dist_to_g_m"] = herit_pts.geometry.distance(g_utm)

# ------------------------------------------------------------
# 3) Save to GPKG and CSV with all attributes
# ------------------------------------------------------------
AMEN_POINTS_GPKG = os.path.join(DATA_DIR, "heritage_points_with_monuments.gpkg")
AMEN_POINTS_CSV  = os.path.join(DATA_DIR, "heritage_points_with_monuments.csv")

# GPKG export (convert odd columns to JSON strings first)
herit_pts_export = _jsonify_weird_columns(herit_pts)
herit_pts_export.to_file(AMEN_POINTS_GPKG, layer="herit_points", driver="GPKG")

# CSV export: use EPSG:4326 + lon/lat + geometry_wkt
herit_ll = herit_pts_export.to_crs("EPSG:4326").copy()
herit_ll["lon"] = herit_ll.geometry.x
herit_ll["lat"] = herit_ll.geometry.y
herit_ll["geometry_wkt"] = herit_ll.geometry.to_wkt()

csv_df = pd.DataFrame(herit_ll.drop(columns=["geometry"]))
csv_df = _jsonify_weird_columns(csv_df)
csv_df.to_csv(AMEN_POINTS_CSV, index=False, encoding="utf-8")

print("Saved points (GPKG):", AMEN_POINTS_GPKG, "n:", len(herit_pts_export))
print("Saved points (CSV): ", AMEN_POINTS_CSV)

# ------------------------------------------------------------
# 4) Folium map with clickable annotations (popups)
# ------------------------------------------------------------
MAP_HTML = os.path.join(DATA_DIR, "heritage_points_map.html")

m = folium.Map(location=[G_LAT, G_LON], zoom_start=14, tiles="CartoDB positron")
folium.Marker(
    location=[G_LAT, G_LON],
    tooltip="Republic Square (g)",
    popup="Republic Square (g)",
    icon=folium.Icon(icon="info-sign")
).add_to(m)

cluster = MarkerCluster(name="Heritage + monuments").add_to(m)

# Choose which fields you want to show in popup
popup_fields = [
    "feature_id",
    "name",
    "historic",
    "heritage",
    "tourism",
    "memorial",
    "artwork_type",
    "wikipedia",
    "wikidata",
    "start_date",
    "addr:street",
    "addr:housenumber",
    "dist_to_g_m",
    "orig_geom_type"
]

herit_ll_iter = herit_ll.copy()
for _, r in herit_ll_iter.iterrows():
    title = _safe_str(r.get("name")) or _safe_str(r.get("historic")) or "Heritage place"

    rows_html = []
    for f in popup_fields:
        if f in r and _safe_str(r[f]) != "":
            val = r[f]
            if f == "dist_to_g_m":
                try:
                    val = f"{float(val):.0f}"
                except Exception:
                    val = _safe_str(val)
            rows_html.append(f"<tr><td><b>{f}</b></td><td>{_safe_str(val)}</td></tr>")

    popup_html = f"""
    <div style="max-width: 360px;">
      <div style="font-size: 14px; font-weight: 600; margin-bottom: 6px;">{title}</div>
      <table style="width: 100%; font-size: 12px;">{''.join(rows_html)}</table>
    </div>
    """

    folium.CircleMarker(
        location=[r["lat"], r["lon"]],
        radius=5,
        fill=True,
        popup=folium.Popup(popup_html, max_width=380),
        tooltip=title
    ).add_to(cluster)

folium.LayerControl().add_to(m)
m.save(MAP_HTML)

print("Saved map HTML:", MAP_HTML)
IFrame(MAP_HTML, width=1100, height=700)


Saved points (GPKG): data/yerevan_full_pipeline\heritage_points_with_monuments.gpkg n: 994
Saved points (CSV):  data/yerevan_full_pipeline\heritage_points_with_monuments.csv
Saved map HTML: data/yerevan_full_pipeline\heritage_points_map.html


In [12]:
# Start from canonical grid
master = grid.copy()

# Merge population
gp = ensure_crs(gpd.read_file(POP_GPKG, layer="grid_pop"), 32638)
master = master.merge(gp[["cell_id","pop_sum","pop_density_per_km2"]], on="cell_id", how="left")

# Merge business
gb = ensure_crs(gpd.read_file(BIZ_GRID_GPKG, layer="grid_biz"), 32638)
master = master.merge(gb[["cell_id","biz_count","biz_density_per_km2"]], on="cell_id", how="left")

# Merge rent
gr = ensure_crs(gpd.read_file(RENT_GRID_GPKG, layer="grid_rent"), 32638)
master = master.merge(gr[["cell_id","rent_listings","implied_rent_per_m2_month"]], on="cell_id", how="left")

# Merge amenity
ga = ensure_crs(gpd.read_file(AMEN_GRID_GPKG, layer="grid_amen"), 32638)
master = master.merge(ga[["cell_id","dist_to_g_m","dist_to_nearest_herit_m","herit_cnt_500m","herit_density_500m"]], on="cell_id", how="left")

# Build final variables
master["cx"] = master.geometry.centroid.x
master["cy"] = master.geometry.centroid.y

master["R_implied"] = pd.to_numeric(master["implied_rent_per_m2_month"], errors="coerce")
master["rent_missing"] = (~np.isfinite(master["R_implied"]) | (master["R_implied"] <= 0)).astype(int)
R_med = np.nanmedian(master.loc[master["rent_missing"] == 0, "R_implied"])
master.loc[master["rent_missing"] == 1, "R_implied"] = R_med
master["log_rent"] = np.log1p(master["R_implied"])

# Use observed M(x)
master["M_obs"] = master["biz_density_per_km2"].fillna(0)

# Define y_business as top 30% business intensity
thr = float(2)
master["y_business"] = (master["M_obs"] >= thr).astype(int)

MASTER_GPKG = os.path.join(DATA_DIR, "master_grid.gpkg")
master.to_file(MASTER_GPKG, layer="master", driver="GPKG")
print("Saved master grid:", MASTER_GPKG)
print("Cells:", len(master), "Business share:", master["y_business"].mean())


Saved master grid: data/yerevan_full_pipeline\master_grid.gpkg
Cells: 1036 Business share: 0.4691119691119691


In [13]:
pd.DataFrame(master).to_csv('master.csv')

In [14]:
master

,cell_id,geometry,area_km2,cx,cy,pop_sum,pop_density_per_km2,biz_count,biz_density_per_km2,rent_listings,implied_rent_per_m2_month,dist_to_g_m,dist_to_nearest_herit_m,herit_cnt_500m,herit_density_500m,R_implied,rent_missing,log_rent,M_obs,y_business
0,17,"POLYGON ((446166.158 4444171.848, 446166.158 4...",0.006514,446124.989309,4.444134e+06,2.814809,432.114292,0,0.0,0,NaN,12847.170533,2893.470419,0,0.0,6.588235,1,2.026599,0.0,0
1,18,"POLYGON ((446166.158 4444171.848, 446072.839 4...",0.060923,445978.826383,4.444288e+06,11.043721,181.274731,0,0.0,0,NaN,12948.067531,2998.921624,0,0.0,6.588235,1,2.026599,0.0,0
2,57,"POLYGON ((446166.158 4444171.848, 446666.158 4...",0.039380,446415.982016,4.444132e+06,19.865776,504.460248,0,0.0,0,NaN,12567.421897,2614.658309,0,0.0,6.588235,1,2.026599,0.0,0
3,58,"POLYGON ((446666.158 4444171.848, 446166.158 4...",0.130023,446416.608316,4.444302e+06,76.684990,589.782013,0,0.0,0,NaN,12521.511533,2568.914426,0,0.0,6.588235,1,2.026599,0.0,0
4,97,"POLYGON ((446666.158 4444171.848, 447166.158 4...",0.040908,446922.502942,4.444131e+06,21.588802,527.743532,0,0.0,0,NaN,12081.520311,2136.821358,0,0.0,6.588235,1,2.026599,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1735,"POLYGON ((467166.158 4443171.848, 467452.13 44...",0.023871,467245.151731,4.443090e+06,1.349567,56.535276,0,0.0,0,NaN,9823.892902,5509.395672,0,0.0,6.588235,1,2.026599,0.0,0
1032,1736,"POLYGON ((467166.158 4443171.848, 467166.158 4...",0.212524,467384.653723,4.443445e+06,16.280460,76.605206,0,0.0,0,NaN,9793.112529,5331.017994,0,0.0,6.588235,1,2.026599,0.0,0
1033,1737,"POLYGON ((467666.158 4443671.848, 467166.158 4...",0.054762,467389.670477,4.443729e+06,3.182314,58.112105,0,0.0,0,NaN,9681.595876,5129.171829,0,0.0,6.588235,1,2.026599,0.0,0
1034,1776,"POLYGON ((467666.158 4443671.848, 467757.713 4...",0.006858,467696.675923,4.443622e+06,0.000000,0.000000,0,0.0,0,NaN,10005.923454,5423.226859,0,0.0,6.588235,1,2.026599,0.0,0


In [15]:
# Initial mu from observed business intensity
w = master["M_obs"].fillna(0).to_numpy()
mu0x = float((master["cx"].to_numpy() * w).sum() / w.sum())
mu0y = float((master["cy"].to_numpy() * w).sum() / w.sum())

def tau_minutes(cx, cy, mux, muy, transport_factor=1.0):
    d = np.sqrt((cx - mux)**2 + (cy - muy)**2)
    return transport_factor * (d / SPEED_M_PER_MIN)

# Baseline tau for estimation
master["tau_min"] = tau_minutes(master["cx"].to_numpy(), master["cy"].to_numpy(), mu0x, mu0y, 1.0)

features = [
    "tau_min",
    "dist_to_g_m",
    "herit_density_500m",
    "pop_density_per_km2",
    "log_rent",
    "rent_missing"
]

est = master[["y_business"] + features].replace([np.inf,-np.inf], np.nan).dropna().copy()
means = est[features].mean()
stds = est[features].std().replace(0, 1.0)

def zscore(dfX):
    Z = dfX.copy()
    for c in dfX.columns:
        Z[c] = (Z[c] - means[c]) / stds[c]
    return Z

X = sm.add_constant(zscore(est[features]), has_constant="add")
y = est["y_business"].astype(int)
logit_model = sm.Logit(y, X).fit(disp=False)
print(logit_model.summary())

def predict_shares(df, mux, muy, transport_factor, amenity_factor):
    tmp = df.copy()
    tmp["tau_min"] = tau_minutes(tmp["cx"].to_numpy(), tmp["cy"].to_numpy(), mux, muy, transport_factor)

    Xraw = tmp[features].replace([np.inf,-np.inf], np.nan)
    Z = zscore(Xraw)

    # Amenity slider scales historic components in standardized space
    Z["dist_to_g_m"] = Z["dist_to_g_m"] * amenity_factor
    Z["herit_density_500m"] = Z["herit_density_500m"] * amenity_factor

    Xmat = sm.add_constant(Z, has_constant="add")
    p = logit_model.predict(Xmat)
    return p.to_numpy()

def solve_mu(df, transport_factor, amenity_factor, mu_init=(mu0x, mu0y), max_iters=30, tol_m=30.0):
    mux, muy = mu_init
    cx = df["cx"].to_numpy()
    cy = df["cy"].to_numpy()

    for _ in range(max_iters):
        s = predict_shares(df, mux, muy, transport_factor, amenity_factor)
        valid = np.isfinite(s)
        s2 = s[valid]
        if s2.sum() <= 1e-9:
            break
        mux_new = float((cx[valid] * s2).sum() / s2.sum())
        muy_new = float((cy[valid] * s2).sum() / s2.sum())
        shift = float(np.sqrt((mux_new - mux)**2 + (muy_new - muy)**2))
        mux, muy = mux_new, muy_new
        if shift < tol_m:
            break

    s_final = predict_shares(df, mux, muy, transport_factor, amenity_factor)
    return (mux, muy), s_final


                           Logit Regression Results                           
Dep. Variable:             y_business   No. Observations:                 1036
Model:                          Logit   Df Residuals:                     1029
Method:                           MLE   Df Model:                            6
Date:                Fri, 06 Feb 2026   Pseudo R-squ.:                  0.4279
Time:                        16:50:35   Log-Likelihood:                -409.71
converged:                       True   LL-Null:                       -716.12
Covariance Type:            nonrobust   LLR p-value:                4.005e-129
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.2523      0.145      1.739      0.082      -0.032       0.537
tau_min                -0.7644      0.754     -1.013      0.311      -2.243       0.714
dist_to_g_m     

In [85]:
# yerevan_sliders_with_single_business_polygon.py
#
# Interactive Leaflet HTML with:
# - Only "hot" cells are filled (share > 0.1)
# - Cell borders are drawn only for hot cells (thin, transparent)
# - Hot fill + borders are moderately transparent (tuned below)
# - City administrative border is thinner
# - Historical center uses custom icon (22x22) with correct anchoring (no drifting on zoom)
# - Business center (μ) is a filled circle
# - Business polygon border is #f0805a fully opaque
# - Legend aligned, includes icons
#
# Output:
#   data/yerevan_interactive/yerevan_sliders_single_business_polygon.html
#
# Assumes you already have in memory:
#   master GeoDataFrame with columns:
#     cell_id, geometry, cx, cy, dist_to_g_m, herit_density_500m,
#     pop_density_per_km2, log_rent, rent_missing
#   logit_model (statsmodels), features list, means, stds, mu0x, mu0y
#   G_LON, G_LAT, SPEED_M_PER_MIN

import os, json, base64, shutil
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, MultiPoint, LineString
from shapely.ops import unary_union
from pyproj import Transformer

OUT_DIR = "data/yerevan_interactive"
os.makedirs(OUT_DIR, exist_ok=True)

# Copy icons into output so HTML can load them via relative paths
ASSETS_SRC = "assets"
ASSETS_DST = os.path.join(OUT_DIR, "assets")
if os.path.isdir(ASSETS_SRC):
    os.makedirs(ASSETS_DST, exist_ok=True)
    for fn in ["icon_historical.png", "icon_business.png"]:
        src = os.path.join(ASSETS_SRC, fn)
        dst = os.path.join(ASSETS_DST, fn)
        if os.path.isfile(src):
            shutil.copy2(src, dst)

def _ensure_crs(gdf: gpd.GeoDataFrame, default_epsg: str = "EPSG:32638") -> gpd.GeoDataFrame:
    if gdf.crs is None:
        gdf = gdf.copy()
        gdf.set_crs(default_epsg, inplace=True)
    return gdf

# -----------------------------
# GeoJSON grid (lightweight)
# -----------------------------
geo = master[["cell_id", "geometry"]].copy().reset_index(drop=True)
geo["gid"] = np.arange(len(geo)).astype(int)

geo = _ensure_crs(geo, "EPSG:32638")
geo_ll = geo[["gid", "geometry"]].to_crs("EPSG:4326")
geo_ll["geometry"] = geo_ll["geometry"].simplify(0.00005, preserve_topology=True)
geojson_obj = json.loads(geo_ll.to_json())

# -----------------------------
# City boundary (approx) from grid union
# -----------------------------
def compute_city_boundary_feature(master_gdf: gpd.GeoDataFrame, simplify_tol_deg: float = 0.00030):
    mg = master_gdf[["geometry"]].copy()
    mg = _ensure_crs(mg, "EPSG:32638").to_crs("EPSG:32638")
    try:
        u = unary_union(list(mg.geometry))
    except Exception:
        u = mg.unary_union

    if u.is_empty:
        return None

    u_ll = gpd.GeoSeries([u], crs="EPSG:32638").to_crs("EPSG:4326").iloc[0]
    if simplify_tol_deg and simplify_tol_deg > 0:
        u_ll = u_ll.simplify(float(simplify_tol_deg), preserve_topology=True)

    return {"type": "Feature", "properties": {}, "geometry": u_ll.__geo_interface__}

city_boundary_feature = compute_city_boundary_feature(master)

# -----------------------------
# Arrays for precompute
# -----------------------------
cx = master["cx"].to_numpy().astype(float)
cy = master["cy"].to_numpy().astype(float)
dist_g = master["dist_to_g_m"].to_numpy().astype(float)
herit = master["herit_density_500m"].fillna(0).to_numpy().astype(float)
pop = master["pop_density_per_km2"].fillna(0).to_numpy().astype(float)
log_rent = master["log_rent"].fillna(master["log_rent"].median()).to_numpy().astype(float)
rent_missing = master["rent_missing"].fillna(1).to_numpy().astype(float)

params = logit_model.params.to_dict()
coef = {
    "const": float(params.get("const", 0.0)),
    "tau_min": float(params.get("tau_min", 0.0)),
    "dist_to_g_m": float(params.get("dist_to_g_m", 0.0)),
    "herit_density_500m": float(params.get("herit_density_500m", 0.0)),
    "pop_density_per_km2": float(params.get("pop_density_per_km2", 0.0)),
    "log_rent": float(params.get("log_rent", 0.0)),
    "rent_missing": float(params.get("rent_missing", 0.0)),
}

m = {k: float(means[k]) for k in features}
s = {k: float(stds[k]) for k in features}

# g point lon/lat and UTM (for separation)
g_lon, g_lat = float(G_LON), float(G_LAT)
g_utm = gpd.GeoSeries([Point(g_lon, g_lat)], crs="EPSG:4326").to_crs("EPSG:32638").iloc[0]
g_utm_x, g_utm_y = float(g_utm.x), float(g_utm.y)

# -----------------------------
# Business polygon settings
# -----------------------------
BUS_TARGET_MASS = 0.55
BUS_MAX_POINTS = 500
BUS_MIN_POINTS = 80

BUS_BUFFER_KM = 0.35
BUS_CLOSE_KM = 0.25
BUS_CORRIDOR_KM = 0.20
BUS_MIN_COMP_MASS = 0.10
BUS_SIMPLIFY_TOL = 0.00025

payload = {
    "geojson": geojson_obj,
    "cx": cx.tolist(),
    "cy": cy.tolist(),
    "dist_g": dist_g.tolist(),
    "herit": herit.tolist(),
    "pop": pop.tolist(),
    "log_rent": log_rent.tolist(),
    "rent_missing": rent_missing.tolist(),
    "coef": coef,
    "mean": m,
    "std": s,
    "mu0": {"x": float(mu0x), "y": float(mu0y)},
    "g": {"lon": float(g_lon), "lat": float(g_lat), "x": float(g_utm_x), "y": float(g_utm_y)},
    "speed_m_per_min": float(SPEED_M_PER_MIN),
    "biz": {
        "target_mass": float(BUS_TARGET_MASS),
        "max_points": int(BUS_MAX_POINTS),
        "min_points": int(BUS_MIN_POINTS),
        "buffer_km": float(BUS_BUFFER_KM),
        "close_km": float(BUS_CLOSE_KM),
        "corridor_km": float(BUS_CORRIDOR_KM),
        "min_component_mass": float(BUS_MIN_COMP_MASS),
        "simplify_tol": float(BUS_SIMPLIFY_TOL),
    },
    "city_boundary": city_boundary_feature,
}

TRANSPORT_STOPS = [0.50, 0.75, 1.00, 1.25, 1.50]
AMENITY_STOPS   = [0.50, 0.75, 1.00, 1.25, 1.50]

# -----------------------------
# Vectorized fixed-point solver
# -----------------------------
def solve_numpy(
    tFactor: float,
    aFactor: float,
    cx: np.ndarray,
    cy: np.ndarray,
    dist_g: np.ndarray,
    herit: np.ndarray,
    pop: np.ndarray,
    log_rent: np.ndarray,
    rent_missing: np.ndarray,
    coef: dict,
    mean: dict,
    std: dict,
    mu0x: float,
    mu0y: float,
    speed_m_per_min: float,
    max_iter: int = 20,
    shift_tol_m: float = 30.0,
):
    def _safe_std(x: float) -> float:
        x = float(x)
        return x if x != 0.0 else 1.0

    mux = float(mu0x)
    muy = float(mu0y)

    eps = 1e-9
    tFactor = float(tFactor)
    aFactor = float(aFactor)

    s_tau = _safe_std(std["tau_min"])
    s_dg  = _safe_std(std["dist_to_g_m"])
    s_h   = _safe_std(std["herit_density_500m"])
    s_p   = _safe_std(std["pop_density_per_km2"])
    s_lr  = _safe_std(std["log_rent"])
    s_rm  = _safe_std(std["rent_missing"])

    z_dg0 = (dist_g - mean["dist_to_g_m"]) / s_dg
    z_h0  = (herit  - mean["herit_density_500m"]) / s_h
    z_p0  = (pop    - mean["pop_density_per_km2"]) / s_p
    z_lr0 = (log_rent - mean["log_rent"]) / s_lr
    z_rm0 = (rent_missing - mean["rent_missing"]) / s_rm

    b0   = float(coef["const"])
    bTau = float(coef["tau_min"])
    bDg  = float(coef["dist_to_g_m"])
    bH   = float(coef["herit_density_500m"])
    bP   = float(coef["pop_density_per_km2"])
    bLR  = float(coef["log_rent"])
    bRM  = float(coef["rent_missing"])

    lin = np.empty_like(cx, dtype=float)

    for _ in range(int(max_iter)):
        dx = cx - mux
        dy = cy - muy
        d = np.hypot(dx, dy)

        tau = (d / float(speed_m_per_min)) / max(tFactor, eps)
        z_tau = (tau - mean["tau_min"]) / s_tau

        lin[:] = (
            b0
            + bTau * z_tau
            + bDg  * (z_dg0 * aFactor)
            + bH   * (z_h0  * aFactor)
            + bP   * z_p0
            + bLR  * z_lr0
            + bRM  * z_rm0
        )

        np.clip(lin, -35.0, 35.0, out=lin)
        shares = 1.0 / (1.0 + np.exp(-lin))

        sumw = float(shares.sum())
        if sumw <= 0 or not np.isfinite(sumw):
            return mux, muy, np.zeros_like(cx, dtype=float)

        mux_new = float((shares * cx).sum() / sumw)
        muy_new = float((shares * cy).sum() / sumw)

        shift = float(np.hypot(mux_new - mux, muy_new - muy))
        mux, muy = mux_new, muy_new

        if shift < float(shift_tol_m):
            break

    return mux, muy, shares

# -----------------------------
# Business polygon from shares
# -----------------------------
def business_polygon_from_shares_python(
    shares: np.ndarray,
    cx: np.ndarray,
    cy: np.ndarray,
    biz_cfg: dict,
):
    s_arr = np.where(np.isfinite(shares), shares, 0.0)
    total_mass = float(s_arr.sum())
    if total_mass <= 0:
        return None, 0.0, 0

    idx = np.argsort(-s_arr)

    target_mass = float(biz_cfg["target_mass"]) * total_mass
    max_points = min(int(biz_cfg["max_points"]), int(len(s_arr)))
    min_points = int(biz_cfg["min_points"])

    selected = []
    cum = 0.0
    for k in range(len(s_arr)):
        if len(selected) >= max_points:
            break
        i = int(idx[k])
        if s_arr[i] <= 0:
            continue
        selected.append(i)
        cum += float(s_arr[i])
        if len(selected) >= min_points and cum >= target_mass:
            break

    used_points = len(selected)
    if used_points < 10:
        return None, 0.0, used_points

    r_m = float(biz_cfg["buffer_km"]) * 1000.0
    close_m = float(biz_cfg["close_km"]) * 1000.0
    corridor_m = float(biz_cfg["corridor_km"]) * 1000.0

    pts = [Point(float(cx[i]), float(cy[i])) for i in selected]
    peak_i = selected[0]
    peak_pt = Point(float(cx[peak_i]), float(cy[peak_i]))

    merged = MultiPoint(pts).buffer(r_m)

    if close_m > 0:
        merged = merged.buffer(close_m).buffer(-close_m)

    if merged.is_empty:
        return None, 0.0, used_points

    if merged.geom_type == "Polygon":
        parts = [merged]
    elif merged.geom_type == "MultiPolygon":
        parts = list(merged.geoms)
    else:
        return None, 0.0, used_points

    if len(parts) > 1:
        comp_mass = []
        for p in parts:
            m0 = 0.0
            for i in selected:
                if p.contains(Point(float(cx[i]), float(cy[i]))):
                    m0 += float(s_arr[i])
            comp_mass.append(m0)

        selected_mass = float(sum(comp_mass))
        min_comp = float(biz_cfg["min_component_mass"]) * selected_mass

        kept = [(parts[i], comp_mass[i]) for i in range(len(parts)) if comp_mass[i] >= min_comp]
        if not kept:
            kept = [(parts[0], comp_mass[0])]

        anchor_poly = None
        for p, _m in kept:
            if p.contains(peak_pt):
                anchor_poly = p
                break
        if anchor_poly is None:
            kept.sort(key=lambda x: x[1], reverse=True)
            anchor_poly = kept[0][0]

        out = anchor_poly
        for p, _m in kept:
            if p.equals(anchor_poly):
                continue
            ca = out.centroid
            cb = p.centroid
            corridor = LineString([ca, cb]).buffer(corridor_m)
            out = unary_union([out, corridor, p])

        if close_m > 0:
            out = out.buffer(close_m).buffer(-close_m)

        merged = out

    area_km2 = float(merged.area) / 1e6
    geom_ll = gpd.GeoSeries([merged], crs="EPSG:32638").to_crs("EPSG:4326").iloc[0]

    simp = float(biz_cfg.get("simplify_tol", 0.0))
    if simp > 0:
        geom_ll = geom_ll.simplify(simp, preserve_topology=True)

    return geom_ll, area_km2, used_points

# -----------------------------
# Shares packing to base64 u8
# -----------------------------
def _shares_to_u8_b64(shares: np.ndarray) -> str:
    x = np.asarray(shares, dtype=float)
    x = np.where(np.isfinite(x), x, 0.0)
    x = np.clip(x, 0.0, 1.0)
    u8 = np.rint(x * 255.0).astype(np.uint8)
    return base64.b64encode(u8.tobytes()).decode("ascii")

def precompute_biz(payload: dict) -> dict:
    cx_np = np.asarray(payload["cx"], dtype=float)
    cy_np = np.asarray(payload["cy"], dtype=float)
    dist_g_np = np.asarray(payload["dist_g"], dtype=float)
    herit_np = np.asarray(payload["herit"], dtype=float)
    pop_np = np.asarray(payload["pop"], dtype=float)
    log_rent_np = np.asarray(payload["log_rent"], dtype=float)
    rent_missing_np = np.asarray(payload["rent_missing"], dtype=float)

    coef0 = payload["coef"]
    mean0 = payload["mean"]
    std0 = payload["std"]

    mu0x0 = float(payload["mu0"]["x"])
    mu0y0 = float(payload["mu0"]["y"])
    speed0 = float(payload["speed_m_per_min"])
    biz_cfg0 = payload["biz"]

    gx = float(payload["g"]["x"])
    gy = float(payload["g"]["y"])

    to_ll = Transformer.from_crs("EPSG:32638", "EPSG:4326", always_xy=True)

    out = {}
    for t in TRANSPORT_STOPS:
        for a in AMENITY_STOPS:
            mux, muy, shares = solve_numpy(
                tFactor=float(t),
                aFactor=float(a),
                cx=cx_np,
                cy=cy_np,
                dist_g=dist_g_np,
                herit=herit_np,
                pop=pop_np,
                log_rent=log_rent_np,
                rent_missing=rent_missing_np,
                coef=coef0,
                mean=mean0,
                std=std0,
                mu0x=mu0x0,
                mu0y=mu0y0,
                speed_m_per_min=speed0,
            )

            sep_m = float(np.hypot(mux - gx, muy - gy))
            lon, lat = to_ll.transform(float(mux), float(muy))

            geom_ll, area_km2, used_pts = business_polygon_from_shares_python(
                shares=shares, cx=cx_np, cy=cy_np, biz_cfg=biz_cfg0
            )

            key = f"{t:.2f}|{a:.2f}"
            rec = {
                "mu": {"x": float(mux), "y": float(muy), "lon": float(lon), "lat": float(lat)},
                "sep_m": float(sep_m),
                "shares_u8_b64": _shares_to_u8_b64(shares),
                "shares_scale": 255,
                "area_km2": 0.0,
                "used_points": int(used_pts),
                "feature": None,
            }
            if geom_ll is not None:
                rec["feature"] = {"type": "Feature", "properties": {}, "geometry": geom_ll.__geo_interface__}
                rec["area_km2"] = float(area_km2) if np.isfinite(area_km2) else 0.0

            out[key] = rec

    return out

def payload_for_browser(payload: dict) -> dict:
    return {
        "geojson": payload["geojson"],
        "g": payload["g"],  # historical center
        "mu0": payload["mu0"],
        "cell_size_m": payload.get("cell_size_m", None),
        "precomputed_biz": payload["precomputed_biz"],
        "city_boundary": payload.get("city_boundary", None),
        "assets": {
            "historical_icon": "assets/icon_historical.png",
            "business_area_icon": "assets/icon_business.png",
        },
    }

# -----------------------------
# HTML builder
# -----------------------------
def build_fast_yerevan_html(payload_browser: dict) -> str:
    payload_json = json.dumps(payload_browser, ensure_ascii=False, separators=(",", ":"))

    return f"""<!doctype html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>Yerevan sliders with single business polygon (fast)</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css"/>
  <style>
    body {{ margin:0; font-family: Arial, sans-serif; }}
    #map {{ height: 86vh; width: 100%; }}
    #controls {{
      height: 14vh; padding: 10px 14px; box-sizing: border-box;
      display: grid; grid-template-columns: 1fr; gap: 8px;
      border-top: 1px solid #ddd;
    }}
    .row {{ display:flex; gap:12px; align-items:center; }}
    .label {{ width: 340px; }}
    .value {{ font-weight: 600; width: 70px; }}
    .info {{ margin-left: 8px; }}
    input[type="range"] {{ width: 460px; }}

    .legend {{
      background: rgba(255,255,255,0.92);
      padding: 10px 12px;
      border-radius: 10px;
      box-shadow: 0 1px 10px rgba(0,0,0,0.12);
      line-height: 1.2;
      color: #111;
      font-size: 13px;
      min-width: 240px;
    }}
    .legend .title {{ font-weight: 700; margin-bottom: 8px; }}
    .legend .item {{
      display:flex;
      align-items:center;
      gap:8px;
      margin: 6px 0;
    }}
    .legend .swatch {{
      width: 14px;
      height: 14px;
      border-radius: 0px;
      border: 1px solid rgba(0,0,0,0.35);
      flex: 0 0 auto;
      margin: 0;
    }}
    .legend .muted {{ color:#444; }}
    .legend .dot {{
      width: 12px;
      height: 12px;
      border-radius: 50%;
      border: 2px solid #111;
      background: #f0805a;
      flex: 0 0 auto;
      margin: 0;
    }}
    .legend img.icon {{
      width: 22px;
      height: 22px;
      object-fit: contain;
      flex: 0 0 auto;
      margin: 0;
      display:block;
    }}
  </style>
</head>
<body>
  <div id="map"></div>
  <div id="controls">
    <div class="row">
      <div class="label">Transport factor (0.1..2.0)</div>
      <div class="value" id="tVal"></div>
      <input id="tSlider" type="range" min="0.10" max="2.00" step="0.01" value="1.00">
      <div class="info">Higher = faster</div>
    </div>

    <div class="row">
      <div class="label">Historic amenity strength (0.1..2.0)</div>
      <div class="value" id="aVal"></div>
      <input id="aSlider" type="range" min="0.10" max="2.00" step="0.01" value="1.00">
      <div class="info">Higher = stronger</div>
    </div>

    <div class="row">
      <div class="label">μ, separation, business area</div>
      <div class="info" id="muInfo"></div>
    </div>
  </div>

  <script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"></script>

  <script>
    const data = {payload_json};

    const COLOR_HOT = "#f0805a";
    const COLOR_LINE = "#4f4f4f";
    const COLOR_CITY = "#3f3f3f";

    // Threshold: share > 0.1
    const THRESH_SHARE = 0.5;
    const THRESH_B = Math.ceil(THRESH_SHARE * 255.0);

    // Thin borders for hot cells only
    const HOT_LINE_WEIGHT = 0.25;

    function b64ToU8(b64) {{
      const bin = atob(b64);
      const len = bin.length;
      const out = new Uint8Array(len);
      for (let i = 0; i < len; i++) out[i] = bin.charCodeAt(i);
      return out;
    }}

    const _sharesCache = new Map();
    function getSharesU8(key) {{
      const hit = _sharesCache.get(key);
      if (hit) {{
        hit.t = performance.now();
        return hit.u8;
      }}
      const rec = data.precomputed_biz && data.precomputed_biz[key];
      if (!rec || !rec.shares_u8_b64) return null;

      const u8 = b64ToU8(rec.shares_u8_b64);
      _sharesCache.set(key, {{ u8, t: performance.now() }});

      if (_sharesCache.size > 4) {{
        let oldestK = null;
        let oldestT = Infinity;
        for (const [k, v] of _sharesCache.entries()) {{
          if (v.t < oldestT) {{ oldestT = v.t; oldestK = k; }}
        }}
        if (oldestK !== null) _sharesCache.delete(oldestK);
      }}
      return u8;
    }}

    // Opacity LUTs for hot cells
    // Requested: "less opacite (less transparent)" => increase alpha
    const FILL_OPACITY_LUT = new Array(256);
    const STROKE_OPACITY_LUT = new Array(256);
    for (let b = 0; b < 256; b++) {{
      if (b <= THRESH_B) {{
        FILL_OPACITY_LUT[b] = 0.0;
        STROKE_OPACITY_LUT[b] = 0.0;
      }} else {{
        const s = b / 255.0;
        const t = Math.min(1.0, Math.max(0.0, (s - THRESH_SHARE) / (1.0 - THRESH_SHARE)));

        // Higher opacity than before
        const fillOp = 0.10 + 0.34 * Math.pow(t, 0.85);     // max about 0.44
        const strokeOp = 0.18 + 0.40 * Math.pow(t, 0.60);   // max about 0.58

        FILL_OPACITY_LUT[b] = Math.min(0.48, Math.max(0.0, fillOp));
        STROKE_OPACITY_LUT[b] = Math.min(0.62, Math.max(0.0, strokeOp));
      }}
    }}

    const g = data.g;

    const map = L.map("map", {{ preferCanvas: true }}).setView([g.lat, g.lon], 11);

    L.tileLayer("https://{{s}}.basemaps.cartocdn.com/light_all/{{z}}/{{x}}/{{y}}{{r}}.png", {{
      maxZoom: 19,
      subdomains: "abcd",
      attribution: '&copy; OpenStreetMap contributors &copy; CARTO'
    }}).addTo(map);

    setTimeout(() => map.invalidateSize(), 0);

    // Historical center icon (22x22) with correct anchoring (no drift on zoom)
    const histIconUrl = (data.assets && data.assets.historical_icon)
      ? data.assets.historical_icon
      : "assets/icon_historical.png";

    const HIST_W = 22;
    const HIST_H = 22;

    const histIcon = L.icon({{
      iconUrl: histIconUrl,
      iconSize: [HIST_W, HIST_H],
      iconAnchor: [HIST_W / 2, HIST_H / 2],
      popupAnchor: [0, -HIST_H / 2]
    }});

    L.marker([g.lat, g.lon], {{ icon: histIcon }})
      .addTo(map)
      .bindPopup("Historical center");

    // Business center (μ) as filled circle
    let muMarker = L.circleMarker([g.lat, g.lon], {{
      radius: 7,
      weight: 2,
      color: "#111",
      fillColor: COLOR_HOT,
      fillOpacity: 1.0
    }}).addTo(map).bindPopup("Business center (μ)");

    // Base style: hide everything for non-hot cells
    const baseStyle = () => {{
      return {{
        fillColor: COLOR_HOT,
        fillOpacity: 0.0,
        color: COLOR_LINE,
        opacity: 0.0,
        weight: 0.0
      }};
    }};

    const gridLayer = L.geoJSON(data.geojson, {{
      style: baseStyle,
      interactive: false,
      renderer: L.canvas({{ padding: 0.5 }})
    }}).addTo(map);

    const nCells = (data.geojson && data.geojson.features) ? data.geojson.features.length : 0;
    const layersByGid = new Array(nCells);
    gridLayer.eachLayer((layer) => {{
      const gid = layer && layer.feature && layer.feature.properties ? layer.feature.properties.gid : null;
      if (Number.isFinite(gid) && gid >= 0 && gid < nCells) layersByGid[gid] = layer;
    }});

    // City boundary overlay (thin)
    if (data.city_boundary && data.city_boundary.geometry) {{
      L.geoJSON(data.city_boundary, {{
        interactive: false,
        style: () => ({{
          color: COLOR_CITY,
          weight: 1.6,
          opacity: 0.70,
          fillOpacity: 0.0
        }})
      }}).addTo(map);
    }}

    // Legend
    const legend = L.control({{ position: "topright" }});
    legend.onAdd = function() {{
      const div = L.DomUtil.create("div", "legend");

      const cellSize = data.cell_size_m;
      const cellTxt = (cellSize && isFinite(cellSize))
        ? `Grid cell: about ${{Math.round(cellSize)}} m on a side.`
        : "Grid cell size in meters is unavailable.";

      const bizAreaIconUrl = (data.assets && data.assets.business_area_icon)
        ? data.assets.business_area_icon
        : "assets/icon_business.png";

      div.innerHTML = `
        <div class="title">Legend</div>

        <div class="item">
          <div class="swatch" style="background:${{COLOR_HOT}};"></div>
          <div>Hot cells (share &gt; 0.1)</div>
        </div>

        <div class="item">
          <div class="swatch" style="background:transparent; border:1px solid rgba(0,0,0,0.45);"></div>
          <div class="muted">${{cellTxt}}</div>
        </div>

        <div class="item" style="margin-top:8px;">
          <img class="icon" src="${{histIconUrl}}" alt="Historical center"/>
          <div class="muted">Historical center</div>
        </div>

        <div class="item">
          <div class="dot"></div>
          <div class="muted">Business center (μ)</div>
        </div>

        <div class="item">
          <img class="icon" src="${{bizAreaIconUrl}}" alt="Business area"/>
          <div class="muted">Business area (polygon)</div>
        </div>
      `;
      L.DomEvent.disableClickPropagation(div);
      L.DomEvent.disableScrollPropagation(div);
      return div;
    }};
    legend.addTo(map);

    // Business polygon layer
    const bizLayer = L.geoJSON(null, {{
      interactive: false,
      style: () => ({{
        color: COLOR_HOT,
        weight: 2.5,
        opacity: 1.0,
        fillColor: COLOR_HOT,
        fillOpacity: 0.10
      }})
    }}).addTo(map);

    function keyTA(tVal, aVal) {{
      return `${{tVal.toFixed(2)}}|${{aVal.toFixed(2)}}`;
    }}

    // Chunked repaint
    let _paintToken = 0;
    function repaintGrid(u8) {{
      const token = ++_paintToken;
      let i = 0;
      const n = layersByGid.length;

      function pump() {{
        if (token !== _paintToken) return;

        const tEnd = performance.now() + 10;
        while (i < n && performance.now() < tEnd) {{
          const layer = layersByGid[i];
          if (layer) {{
            const b = u8[i] || 0;
            if (b <= THRESH_B) {{
              layer.setStyle({{
                fillOpacity: 0.0,
                opacity: 0.0,
                weight: 0.0
              }});
            }} else {{
              layer.setStyle({{
                fillColor: COLOR_HOT,
                fillOpacity: FILL_OPACITY_LUT[b],
                color: COLOR_LINE,
                opacity: STROKE_OPACITY_LUT[b],
                weight: HOT_LINE_WEIGHT
              }});
            }}
          }}
          i++;
        }}
        if (i < n) requestAnimationFrame(pump);
      }}

      requestAnimationFrame(pump);
    }}

    let _lastKey = null;

    function applyState(key) {{
      const rec = data.precomputed_biz && data.precomputed_biz[key];
      if (!rec) return;

      const mu = rec.mu;
      if (mu && isFinite(mu.lat) && isFinite(mu.lon)) {{
        muMarker.setLatLng([mu.lat, mu.lon]);
      }}

      bizLayer.clearLayers();
      if (rec.feature) bizLayer.addData(rec.feature);

      const u8 = getSharesU8(key);
      if (u8) repaintGrid(u8);

      const areaKm2 = (rec.area_km2 && isFinite(rec.area_km2)) ? rec.area_km2 : 0.0;
      const sep = (rec.sep_m && isFinite(rec.sep_m)) ? rec.sep_m : 0.0;

      const muInfoEl = document.getElementById("muInfo");
      if (muInfoEl) {{
        muInfoEl.textContent =
          `μ: (${{(mu && mu.lon)? mu.lon.toFixed(5): "?"}}, ${{(mu && mu.lat)? mu.lat.toFixed(5): "?"}}) | |μ-g|: ${{sep.toFixed(0)}} m | business area: ${{areaKm2.toFixed(2)}} km²`;
      }}
    }}

    let _raf = 0;
    function update() {{
      const tVal = parseFloat(document.getElementById("tSlider").value);
      const aVal = parseFloat(document.getElementById("aSlider").value);

      const tEl = document.getElementById("tVal");
      const aEl = document.getElementById("aVal");
      if (tEl) tEl.textContent = tVal.toFixed(2);
      if (aEl) aEl.textContent = aVal.toFixed(2);

      const key = keyTA(tVal, aVal);
      if (key === _lastKey) return;
      _lastKey = key;

      applyState(key);
    }}

    function scheduleUpdate() {{
      if (_raf) cancelAnimationFrame(_raf);
      _raf = requestAnimationFrame(() => {{
        _raf = 0;
        update();
      }});
    }}

    document.getElementById("tSlider").addEventListener("input", scheduleUpdate);
    document.getElementById("aSlider").addEventListener("input", scheduleUpdate);

    update();
  </script>
</body>
</html>
"""

# -----------------------------
# Approx cell size for legend
# -----------------------------
try:
    master0 = _ensure_crs(master, "EPSG:32638")
    cell0 = master0["geometry"].iloc[0]
    minx, miny, maxx, maxy = cell0.bounds
    payload["cell_size_m"] = float(max(maxx - minx, maxy - miny))
except Exception:
    payload["cell_size_m"] = None

# -----------------------------
# Precompute, build, write
# -----------------------------
payload["precomputed_biz"] = precompute_biz(payload)
payload_js = payload_for_browser(payload)

html_filled = build_fast_yerevan_html(payload_js)
html_path = os.path.join(OUT_DIR, "yerevan_sliders_single_business_polygon.html")
with open(html_path, "w", encoding="utf-8") as f:
    f.write(html_filled)

html_path


'data/yerevan_interactive\\yerevan_sliders_single_business_polygon.html'

In [17]:
# yerevan_business_polygon_precompute.py
#
# Output:
#   data/yerevan_interactive/yerevan_business_polygon_precomputed.html
#
# Assumes you already have:
# - master GeoDataFrame with columns: cx, cy, dist_to_g_m, herit_density_500m, pop_density_per_km2, log_rent, rent_missing
# - logit_model (statsmodels), features list, means, stds, mu0x, mu0y
# - G_LON, G_LAT, SPEED_M_PER_MIN
#
# Notes:
# - sliders snap to precomputed grid (T_STEP, A_STEP).
# - no grid, no tiles. Only polygon + μ and g markers.
#
# Key change vs your version:
# - payload is JSON-minified, gzip-compressed, base64-embedded
# - HTML inflates it in-browser with pako (gzip)
# This keeps polygon count and geometry unchanged, but shrinks HTML a lot.

import os
import json
import gzip
import base64
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString, mapping
from shapely.ops import unary_union, transform
from shapely.prepared import prep
from pyproj import Transformer
from tqdm import tqdm

OUT_DIR = "data/yerevan_interactive"
os.makedirs(OUT_DIR, exist_ok=True)

# -----------------------------
# Input arrays
# -----------------------------
cx = master["cx"].to_numpy().astype(float)  # UTM meters
cy = master["cy"].to_numpy().astype(float)

dist_g = master["dist_to_g_m"].to_numpy().astype(float)
herit = master["herit_density_500m"].fillna(0).to_numpy().astype(float)
pop = master["pop_density_per_km2"].fillna(0).to_numpy().astype(float)
log_rent = master["log_rent"].fillna(master["log_rent"].median()).to_numpy().astype(float)
rent_missing = master["rent_missing"].fillna(1).to_numpy().astype(float)

params = logit_model.params.to_dict()
coef = {
    "const": float(params.get("const", 0.0)),
    "tau_min": float(params.get("tau_min", 0.0)),
    "dist_to_g_m": float(params.get("dist_to_g_m", 0.0)),
    "herit_density_500m": float(params.get("herit_density_500m", 0.0)),
    "pop_density_per_km2": float(params.get("pop_density_per_km2", 0.0)),
    "log_rent": float(params.get("log_rent", 0.0)),
    "rent_missing": float(params.get("rent_missing", 0.0)),
}
mean = {k: float(means[k]) for k in features}
std = {k: float(stds[k]) for k in features}

speed_m_per_min = float(SPEED_M_PER_MIN)

# g in lon/lat and UTM
g_lon, g_lat = float(G_LON), float(G_LAT)
g_utm = gpd.GeoSeries([Point(g_lon, g_lat)], crs="EPSG:4326").to_crs("EPSG:32638").iloc[0]
g_utm_x, g_utm_y = float(g_utm.x), float(g_utm.y)

mu0x = float(mu0x)
mu0y = float(mu0y)

# -----------------------------
# Business polygon settings
# -----------------------------
BUS_TARGET_MASS = 0.55
BUS_MAX_POINTS = 1300
BUS_MIN_POINTS = 80

BUS_BUFFER_KM = 0.35
BUS_CLOSE_KM = 0.25
BUS_CORRIDOR_KM = 0.20
BUS_MIN_COMP_MASS = 0.10

BUS_SIMPLIFY_TOL_DEG = 0.00025  # simplify after projecting to EPSG:4326

# -----------------------------
# Precompute grid (snap-to)
# -----------------------------
T_MIN, T_MAX, T_STEP = 0.10, 2.00, 0.05
A_MIN, A_MAX, A_STEP = 0.10, 2.00, 0.05

t_vals = np.round(np.arange(T_MIN, T_MAX + 1e-9, T_STEP), 2)
a_vals = np.round(np.arange(A_MIN, A_MAX + 1e-9, A_STEP), 2)

# -----------------------------
# Helpers
# -----------------------------
def sigmoid(x: np.ndarray) -> np.ndarray:
    x = np.clip(x, -35, 35)
    return 1.0 / (1.0 + np.exp(-x))

def z(x: np.ndarray, m: float, s: float) -> np.ndarray:
    if not np.isfinite(s) or s == 0:
        return np.zeros_like(x, dtype=float)
    out = (x - m) / s
    out[~np.isfinite(out)] = 0.0
    return out

def solve_mu_and_shares(t_factor: float, a_factor: float, iters: int = 20, tol_m: float = 30.0):
    mux, muy = mu0x, mu0y
    shares = np.zeros_like(cx, dtype=float)

    for _ in range(iters):
        dx = cx - mux
        dy = cy - muy
        d = np.sqrt(dx * dx + dy * dy)
        tau = t_factor * (d / speed_m_per_min)

        z_tau = z(tau, mean["tau_min"], std["tau_min"])
        z_dg = z(dist_g, mean["dist_to_g_m"], std["dist_to_g_m"]) * a_factor
        z_herit = z(herit, mean["herit_density_500m"], std["herit_density_500m"]) * a_factor
        z_pop = z(pop, mean["pop_density_per_km2"], std["pop_density_per_km2"])
        z_lr = z(log_rent, mean["log_rent"], std["log_rent"])
        z_rm = z(rent_missing, mean["rent_missing"], std["rent_missing"])

        lin = (
            coef["const"]
            + coef["tau_min"] * z_tau
            + coef["dist_to_g_m"] * z_dg
            + coef["herit_density_500m"] * z_herit
            + coef["pop_density_per_km2"] * z_pop
            + coef["log_rent"] * z_lr
            + coef["rent_missing"] * z_rm
        )
        shares = sigmoid(lin)

        sumw = shares.sum()
        if sumw <= 0:
            break

        mux_new = float((shares * cx).sum() / sumw)
        muy_new = float((shares * cy).sum() / sumw)
        shift = float(np.hypot(mux_new - mux, muy_new - muy))

        mux, muy = mux_new, muy_new
        if shift < tol_m:
            break

    return mux, muy, shares

def _explode_polys(geom):
    if geom is None or geom.is_empty:
        return []
    gt = geom.geom_type
    if gt == "Polygon":
        return [geom]
    if gt == "MultiPolygon":
        return list(geom.geoms)
    if gt == "GeometryCollection":
        out = []
        for g in geom.geoms:
            out.extend(_explode_polys(g))
        return out
    return []

# Project UTM (EPSG:32638) -> lonlat (EPSG:4326)
to_ll = Transformer.from_crs("EPSG:32638", "EPSG:4326", always_xy=True).transform

def business_polygon_from_shares_one(shares: np.ndarray):
    n = shares.size
    s_clean = np.where(np.isfinite(shares) & (shares > 0), shares, 0.0)
    total_mass = float(s_clean.sum())
    if total_mass <= 0:
        return None, 0.0, 0

    idx = np.argsort(-s_clean)  # desc
    target_mass = BUS_TARGET_MASS * total_mass

    selected = []
    cum = 0.0
    max_points = min(BUS_MAX_POINTS, n)

    for i in idx[:max_points]:
        si = float(s_clean[i])
        if si <= 0:
            continue
        selected.append(int(i))
        cum += si
        if len(selected) >= BUS_MIN_POINTS and cum >= target_mass:
            break

    if len(selected) < 10:
        return None, 0.0, len(selected)

    # Buffers in UTM meters
    r_m = BUS_BUFFER_KM * 1000.0
    close_m = BUS_CLOSE_KM * 1000.0
    corridor_m = BUS_CORRIDOR_KM * 1000.0

    pts = [Point(float(cx[i]), float(cy[i])) for i in selected]
    buffered = [p.buffer(r_m, resolution=16) for p in pts]
    merged = unary_union(buffered)
    if merged is None or merged.is_empty:
        return None, 0.0, len(selected)

    # Closing to merge near blobs
    try:
        merged = merged.buffer(close_m, resolution=16).buffer(-close_m, resolution=16)
    except Exception:
        pass

    parts = _explode_polys(merged)

    if len(parts) > 1:
        comp_mass = [0.0] * len(parts)
        prepped = [prep(p) for p in parts]
        for i in selected:
            p = Point(float(cx[i]), float(cy[i]))
            si = float(s_clean[i])
            for ci, pr in enumerate(prepped):
                if pr.contains(p) or pr.covers(p):
                    comp_mass[ci] += si
                    break

        min_comp = BUS_MIN_COMP_MASS * float(cum)
        kept = [(parts[i], comp_mass[i]) for i in range(len(parts)) if comp_mass[i] >= min_comp]
        if not kept:
            kept = [(parts[0], comp_mass[0])]

        peak_i = int(selected[0])
        peak_pt = Point(float(cx[peak_i]), float(cy[peak_i]))

        anchor_geom = None
        for pg, _m in kept:
            pr = prep(pg)
            if pr.contains(peak_pt) or pr.covers(peak_pt):
                anchor_geom = pg
                break
        if anchor_geom is None:
            kept.sort(key=lambda x: x[1], reverse=True)
            anchor_geom = kept[0][0]

        out = anchor_geom
        out_cent = out.centroid

        for pg, _m in kept:
            if pg.equals(anchor_geom):
                continue
            line = LineString([out_cent.coords[0], pg.centroid.coords[0]])
            corridor = line.buffer(corridor_m, resolution=8)
            out = unary_union([out, corridor, pg])
            out_cent = out.centroid

        try:
            out = out.buffer(close_m, resolution=16).buffer(-close_m, resolution=16)
        except Exception:
            pass

        merged = out

    # Fix invalids
    try:
        merged = merged.buffer(0)
    except Exception:
        pass

    # Area in km2 (UTM meters)
    area_km2 = float(merged.area) / 1e6 if merged and not merged.is_empty else 0.0

    # Project to lon/lat and simplify in degrees
    merged_ll = transform(to_ll, merged)
    if BUS_SIMPLIFY_TOL_DEG and BUS_SIMPLIFY_TOL_DEG > 0:
        try:
            merged_ll = merged_ll.simplify(BUS_SIMPLIFY_TOL_DEG, preserve_topology=True)
        except Exception:
            pass

    geom_json = mapping(merged_ll) if merged_ll and not merged_ll.is_empty else None
    return geom_json, area_km2, len(selected)

def utm_point_to_ll(x: float, y: float):
    lon, lat = to_ll(x, y)
    return float(lon), float(lat)

# -----------------------------
# Precompute table
# -----------------------------
precomp = {}
total = len(t_vals) * len(a_vals)

for t in tqdm(t_vals, total=len(t_vals), desc="t grid"):
    for a in a_vals:
        mux, muy, shares = solve_mu_and_shares(float(t), float(a))
        geom_json, area_km2, used_pts = business_polygon_from_shares_one(shares)

        mu_lon, mu_lat = utm_point_to_ll(mux, muy)
        sep_m = float(np.hypot(mux - g_utm_x, muy - g_utm_y))

        key = f"{t:.2f}_{a:.2f}"
        precomp[key] = {
            # Keep only what the page uses. This does not affect polygon geometry.
            "mu": {"lon": mu_lon, "lat": mu_lat},
            "sep_m": sep_m,
            "area_km2": float(area_km2),
            "poly_geom": geom_json,  # GeoJSON geometry (Polygon or MultiPolygon), holes preserved
        }

payload = {
    "t_grid": {"min": T_MIN, "max": T_MAX, "step": T_STEP},
    "a_grid": {"min": A_MIN, "max": A_MAX, "step": A_STEP},
    "g": {"lon": g_lon, "lat": g_lat},
    "precomp": precomp,
}

# -----------------------------
# HTML
# -----------------------------
html_template = r"""
<!doctype html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>Yerevan business polygon (precomputed)</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css"/>
  <style>
    body { margin:0; font-family: Arial, sans-serif; }
    #map { height: 86vh; width: 100%; }
    #controls {
      height: 14vh; padding: 10px 14px; box-sizing: border-box;
      display: grid; grid-template-columns: 1fr; gap: 8px;
      border-top: 1px solid #ddd;
      background: #fff;
    }
    .row { display:flex; gap:12px; align-items:center; }
    .label { width: 340px; }
    .value { font-weight: 600; width: 70px; }
    .info { margin-left: 8px; }
    input[type="range"] { width: 460px; }
    .leaflet-container { background: #fff; }
  </style>
</head>
<body>
  <div id="map"></div>
  <div id="controls">
    <div class="row">
      <div class="label">Transport factor (snap grid)</div>
      <div class="value" id="tVal"></div>
      <input id="tSlider" type="range" min="0.10" max="2.00" step="0.01" value="1.00">
      <div class="info" id="tSnap"></div>
    </div>

    <div class="row">
      <div class="label">Historic amenity strength (snap grid)</div>
      <div class="value" id="aVal"></div>
      <input id="aSlider" type="range" min="0.10" max="2.00" step="0.01" value="1.00">
      <div class="info" id="aSnap"></div>
    </div>

    <div class="row">
      <div class="label">μ, separation, business area</div>
      <div class="info" id="muInfo"></div>
    </div>
  </div>

  <script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/pako@2.1.0/dist/pako.min.js"></script>

  <script>
    // Payload is gzip-compressed and base64-embedded to keep this HTML small.
    const PAYLOAD_GZ_B64 = "__PAYLOAD_GZ_B64__";

    function loadPayload() {
      const bin = Uint8Array.from(atob(PAYLOAD_GZ_B64), c => c.charCodeAt(0));
      const jsonStr = pako.ungzip(bin, { to: "string" });
      return JSON.parse(jsonStr);
    }

    const data = loadPayload();
    const g = data.g;
    const precomp = data.precomp;
    const tg = data.t_grid;
    const ag = data.a_grid;

    function clamp(x, lo, hi) { return Math.max(lo, Math.min(hi, x)); }

    function snap(val, grid) {
      const v = clamp(val, grid.min, grid.max);
      const k = Math.round((v - grid.min) / grid.step);
      const snapped = grid.min + k * grid.step;
      return Number(snapped.toFixed(2));
    }

    function keyFor(t, a) {
      return `${t.toFixed(2)}_${a.toFixed(2)}`;
    }

    // map (no tiles)
    const map = L.map("map", { zoomControl: true, attributionControl: false }).setView([g.lat, g.lon], 12);

    const gMarker = L.circleMarker([g.lat, g.lon], {
      radius: 6, weight: 2, color: "#000", fillColor: "#000", fillOpacity: 1
    }).addTo(map).bindPopup("g (historic center)");

    let muMarker = L.circleMarker([g.lat, g.lon], {
      radius: 7, weight: 2, color: "#000", fillColor: "#fff", fillOpacity: 1
    }).addTo(map).bindPopup("μ (business center)");

    let bizLayer = null;
    let didFit = false;

    function setPolygon(entry) {
      const mu = entry.mu;
      muMarker.setLatLng([mu.lat, mu.lon]);

      if (bizLayer) { map.removeLayer(bizLayer); bizLayer = null; }

      if (entry.poly_geom) {
        const feat = { "type": "Feature", "properties": {}, "geometry": entry.poly_geom };
        bizLayer = L.geoJSON(feat, {
          style: () => ({ color:"#000", weight:2, opacity:0.95, fillColor:"#000", fillOpacity:0.12 })
        }).addTo(map);

        if (!didFit) {
          try { map.fitBounds(bizLayer.getBounds(), { padding: [20, 20] }); } catch (e) {}
          didFit = true;
        }
      }

      document.getElementById("muInfo").textContent =
        `μ: (${mu.lon.toFixed(5)}, ${mu.lat.toFixed(5)}) | |μ-g|: ${entry.sep_m.toFixed(0)} m | area: ${entry.area_km2.toFixed(2)} km²`;
    }

    function update() {
      const tRaw = parseFloat(document.getElementById("tSlider").value);
      const aRaw = parseFloat(document.getElementById("aSlider").value);

      document.getElementById("tVal").textContent = tRaw.toFixed(2);
      document.getElementById("aVal").textContent = aRaw.toFixed(2);

      const tS = snap(tRaw, tg);
      const aS = snap(aRaw, ag);

      document.getElementById("tSnap").textContent = `snap: ${tS.toFixed(2)}`;
      document.getElementById("aSnap").textContent = `snap: ${aS.toFixed(2)}`;

      const key = keyFor(tS, aS);
      const entry = precomp[key];
      if (entry) setPolygon(entry);
    }

    let pending = null;
    function scheduleUpdate() {
      if (pending) clearTimeout(pending);
      pending = setTimeout(update, 30);
    }

    document.getElementById("tSlider").addEventListener("input", scheduleUpdate);
    document.getElementById("aSlider").addEventListener("input", scheduleUpdate);

    update();
  </script>
</body>
</html>
"""

# -----------------------------
# Build embedded payload (minify JSON, gzip, base64)
# -----------------------------
payload_json = json.dumps(payload, separators=(",", ":"), ensure_ascii=False).encode("utf-8")
payload_gz = gzip.compress(payload_json, compresslevel=9)
payload_b64 = base64.b64encode(payload_gz).decode("ascii")

html_filled = html_template.replace("__PAYLOAD_GZ_B64__", payload_b64)

html_path = os.path.join(OUT_DIR, "yerevan_business_polygon_precomputed.html")
with open(html_path, "w", encoding="utf-8") as f:
    f.write(html_filled)

html_path


t grid: 100%|██████████████████████████████████████████████████████████████████████████| 39/39 [04:12<00:00,  6.48s/it]


'data/yerevan_interactive\\yerevan_business_polygon_precomputed.html'

In [79]:
from pathlib import Path
from IPython.display import IFrame, HTML, display

def write_interactive_html(out_path="thesis_dashboard_fit_one_screen.html"):
    html = r"""<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <title>Interactive thesis figures (3-up + compact controls)</title>

  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">

  <script src="https://cdn.plot.ly/plotly-2.30.0.min.js"></script>

  <style>
    :root{
      --pad: 12px;
      --gap: 10px;
      --line: rgba(0,0,0,0.12);
      --muted: rgba(0,0,0,0.62);

      /* Golden mean: cap plot heights so they do not stretch */
      --plotH: clamp(220px, 34vh, 320px);
      --plotHSmall: clamp(200px, 30vh, 280px);
    }

    html, body { height: 100%; }
    body {
      font-family: Inter, system-ui, -apple-system, Segoe UI, Roboto, sans-serif;
      margin: var(--pad);
      background: #fff;
      color: #111;
    }

    h2 { margin: 0 0 8px 0; font-size: 18px; }

    .grid3 {
      display: grid;
      grid-template-columns: repeat(3, minmax(240px, 1fr));
      gap: var(--gap);
      align-items: start;
      margin-bottom: var(--gap);
    }

    .card {
      border: 1px solid var(--line);
      border-radius: 14px;
      padding: 10px;
      background: #fff;
    }

    .plotTitle { font-weight: 750; margin: 0 0 4px 0; font-size: 13px; }
    .small { color: var(--muted); font-size: 11px; line-height: 1.25; margin: 0 0 6px 0; }
    .pill { display:inline-block; padding: 1px 7px; border: 1px solid var(--line); border-radius: 999px; font-size: 11px; margin-left: 6px; }

    /* Key change: explicit heights, not flex-fill */
    .plot { height: var(--plotH); }
    .plotSmall { height: var(--plotHSmall); }

    .controls {
      display: grid;
      grid-template-columns: 1fr 260px;
      gap: 12px;
      align-items: start;
    }

    .sliderRow {
      display: grid;
      grid-template-columns: 24px 1fr 72px;
      gap: 10px;
      align-items: center;
      margin: 6px 0;
    }
    .sliderRow .lbl { font-size: 12px; font-weight: 650; }
    .sliderRow .val { font-size: 12px; text-align: right; color: #222; }
    input[type="range"] { width: 100%; }

    .kvs { font-size: 12px; line-height: 1.25; }
    .kvs b { font-weight: 650; }
    hr { border: 0; border-top: 1px solid rgba(0,0,0,0.08); margin: 8px 0; }

    @media (max-width: 1050px) {
      .grid3 { grid-template-columns: 1fr; }
      .controls { grid-template-columns: 1fr; }
      :root{
        --plotH: clamp(240px, 38vh, 360px);
        --plotHSmall: clamp(220px, 34vh, 320px);
      }
    }
  </style>
</head>

<body>
  <h2>Interactive figures: 3 plots in a row, compact sliders below</h2>

  <div class="grid3">
    <div class="card">
      <div class="plotTitle">City layout <span class="pill">instant response</span></div>
      <div class="small">
        Thick segment is the business area. The x marker is the historic center (g). The circle is the economic center (μ).
      </div>
      <div id="plot_layout" class="plot plotSmall"></div>
    </div>

    <div class="card">
      <div class="plotTitle">Figure 1 style: sweep historic center location</div>
      <div class="small">
        We sweep g from 0 to 1.0 (x axis). Dashed line is the current g from the slider.
        Green X markers show current q and p at that g.
      </div>
      <div id="plot_fig1" class="plot"></div>
    </div>

    <div class="card">
      <div class="plotTitle">Figure 2-3 style: sweep transport cost</div>
      <div class="small">
        We sweep t from 0.0 to 1.5 (y axis). Dashed vertical line marks g. Dots show the current t.
      </div>
      <div id="plot_fig2" class="plot"></div>
    </div>
  </div>

  <div class="card">
    <div class="controls">
      <div>
        <div class="plotTitle">Common parameters</div>
        <div class="small">These sliders update all three plots.</div>

        <div class="sliderRow">
          <div class="lbl">N</div>
          <input id="N" type="range" min="0.20" max="1.90" step="0.05" value="1.40"/>
          <div class="val" id="N_val"></div>
        </div>

        <div class="sliderRow">
          <div class="lbl">e</div>
          <input id="e" type="range" min="0.01" max="1.00" step="0.01" value="0.10"/>
          <div class="val" id="e_val"></div>
        </div>

        <div class="sliderRow">
          <div class="lbl">g</div>
          <input id="g" type="range" min="0.00" max="1.00" step="0.01" value="0.25"/>
          <div class="val" id="g_val"></div>
        </div>

        <div class="sliderRow">
          <div class="lbl">t</div>
          <input id="t" type="range" min="0.00" max="1.50" step="0.01" value="0.25"/>
          <div class="val" id="t_val"></div>
        </div>

        <div class="small" style="margin-top:6px;">
          Tip: Increasing t makes distance more painful, so the business area often shifts to reduce travel.
        </div>
      </div>

      <div>
        <div class="plotTitle">Current interpretation</div>
        <div class="kvs">
          <div style="margin-top:4px;"><b>Business area</b></div>
          <div>Left edge: <span id="q_out"></span></div>
          <div>Right edge: <span id="p_out"></span></div>

          <div style="margin-top:6px;"><b>Historic center</b>: <span id="g_out"></span></div>
          <div><b>Economic center</b>: <span id="mu_out"></span></div>

          <hr/>

          <div><b>Situation label</b></div>
          <div id="case_out" class="small" style="margin:0;"></div>
        </div>
      </div>
    </div>
  </div>

<script>
  const EPS = 1e-9;

  function clip(x, lo, hi) { return Math.max(lo, Math.min(hi, x)); }
  function fmt(x) {
    if (!isFinite(x)) return "nan";
    const ax = Math.abs(x);
    if (ax >= 10) return x.toFixed(3);
    return x.toFixed(4);
  }

  function elH(id, fallback){
    const el = document.getElementById(id);
    const h = el ? el.clientHeight : 0;
    return (h && h > 50) ? h : fallback;
  }

  function solveBounds(N, t, e, g) {
    if (!(N > 0 && N < 2)) return { ok:false, reason:"N must be between 0 and 2." };
    if (!(e > 0)) return { ok:false, reason:"e must be positive." };
    if (!(t >= 0)) return { ok:false, reason:"t must be zero or positive." };

    const M = 2 - N;
    const theta = t / e;

    function inCity(q, p) {
      return (q < p) && (q >= -1 - 1e-6) && (p <= 1 + 1e-6);
    }

    let muB = NaN, pB = NaN, qB = NaN;
    if (theta > EPS) {
      muB = -M * N / (4 * theta);
      pB = muB + M / 2;
      qB = muB - M / 2;
    }

    let muC = NaN, pC = NaN, qC = NaN;
    if (theta > EPS) {
      muC =  M * N / (4 * theta);
      pC = muC + M / 2;
      qC = muC - M / 2;
    }

    let muA = NaN, pA = NaN, qA = NaN;
    const denom = (N - 2 * theta);
    if (Math.abs(denom) > EPS) {
      muA = N * g / denom;
      pA = muA + M / 2;
      qA = muA - M / 2;
    }

    function scoreCase(kind, q, p, mu) {
      if (!isFinite(q) || !isFinite(p) || !isFinite(mu)) return null;
      let score = 0.0;

      if (!inCity(q, p)) {
        score += 10.0 * (Math.max(0.0, (-1 - q)) + Math.max(0.0, (p - 1)));
      }
      if (kind === "A") {
        score += Math.max(0.0, q - g) + Math.max(0.0, g - p);
      } else if (kind === "B") {
        score += Math.max(0.0, p - g);
      } else if (kind === "C") {
        score += Math.max(0.0, g - q);
      }
      return score;
    }

    const candidates = [];
    const sA = scoreCase("A", qA, pA, muA);
    if (sA !== null) candidates.push([sA, "Historic center sits inside the business area.", qA, pA, muA]);
    const sB = scoreCase("B", qB, pB, muB);
    if (sB !== null) candidates.push([sB, "Historic center is to the right of the business area.", qB, pB, muB]);
    const sC = scoreCase("C", qC, pC, muC);
    if (sC !== null) candidates.push([sC, "Historic center is to the left of the business area.", qC, pC, muC]);

    if (candidates.length === 0) return { ok:false, reason:"No valid configuration found for these values." };

    candidates.sort((a,b) => a[0] - b[0]);
    const best = candidates[0];

    return {
      ok: true,
      N, M, t, e, g, theta,
      caseText: best[1],
      q: best[2],
      p: best[3],
      mu: best[4]
    };
  }

  const state = { N: 1.40, e: 0.10, g: 0.25, t: 0.25 };

  function readSlider(id) {
    return parseFloat(document.getElementById(id).value);
  }

  function syncLabelsFromState() {
    document.getElementById("N_val").textContent = fmt(state.N);
    document.getElementById("e_val").textContent = fmt(state.e);
    document.getElementById("g_val").textContent = fmt(state.g);
    document.getElementById("t_val").textContent = fmt(state.t);
  }

  function updateText(sol) {
    if (!sol.ok) {
      document.getElementById("q_out").textContent = "nan";
      document.getElementById("p_out").textContent = "nan";
      document.getElementById("g_out").textContent = "nan";
      document.getElementById("mu_out").textContent = "nan";
      document.getElementById("case_out").textContent = sol.reason;
      return;
    }
    document.getElementById("q_out").textContent = fmt(sol.q);
    document.getElementById("p_out").textContent = fmt(sol.p);
    document.getElementById("g_out").textContent = fmt(sol.g);
    document.getElementById("mu_out").textContent = fmt(sol.mu);
    document.getElementById("case_out").textContent = sol.caseText;
  }

  function drawLayout(sol) {
    let q = NaN, p = NaN, mu = NaN, g = NaN;
    if (sol.ok) { q = sol.q; p = sol.p; mu = sol.mu; g = sol.g; }

    const qd = clip(q, -1, 1);
    const pd = clip(p, -1, 1);

    const data = [
      { x: [-1, 1], y: [0, 0], mode: "lines", line: { width: 6 }, name: "city" },
      { x: sol.ok ? [qd, pd] : [], y: sol.ok ? [0, 0] : [], mode: "lines",
        line: { width: 18 }, opacity: 0.25, name: "business area" },
      { x: sol.ok ? [g] : [], y: sol.ok ? [0] : [], mode: "markers+text",
        marker: { size: 12, symbol: "x" }, text: ["g"], textposition: "top center", name: "historic center" },
      { x: sol.ok ? [mu] : [], y: sol.ok ? [0] : [], mode: "markers+text",
        marker: { size: 10, symbol: "circle" }, text: ["μ"], textposition: "top center", name: "economic center" },
      { x: sol.ok ? [q, p] : [], y: sol.ok ? [0, 0] : [], mode: "markers+text",
        marker: { size: 14, symbol: "line-ns-open" }, text: ["q", "p"], textposition: "bottom center", name: "edges" }
    ];

    const layout = {
      font: { size: 11 },
      margin: { l: 34, r: 10, t: 4, b: 26 },
      xaxis: { range: [-1.05, 1.05], title: "location on the city line" },
      yaxis: { range: [-0.8, 0.8], visible: false },
      showlegend: false,
      height: elH("plot_layout", 260)
    };

    Plotly.react("plot_layout", data, layout, {displayModeBar: false, responsive: false});
  }

  function drawFig1() {
    const N = state.N, e = state.e, t = state.t;
    const gmax = 1.0;
    const n = 320;

    const gs = [];
    const qs = [];
    const ps = [];

    for (let i=0; i<n; i++) {
      const g = gmax * (i/(n-1));
      const sol = solveBounds(N, t, e, g);
      gs.push(g);
      qs.push(sol.ok ? sol.q : NaN);
      ps.push(sol.ok ? sol.p : NaN);
    }

    const gCur = state.g;
    const solCur = solveBounds(N, t, e, gCur);
    const qCur = solCur.ok ? solCur.q : NaN;
    const pCur = solCur.ok ? solCur.p : NaN;

    const yLo = -1.05, yHi = 1.05;

    const data = [
      { x: gs, y: gs, mode: "lines", name: "g (reference)" },
      { x: gs, y: ps, mode: "lines", name: "right edge p(g)" },
      { x: gs, y: qs, mode: "lines", name: "left edge q(g)" },

      { x: gs, y: qs, mode: "lines", line: { width: 0 }, showlegend: false },
      { x: gs, y: ps, mode: "lines", fill: "tonexty", opacity: 0.2, name: "business area" },

      { x: [gCur, gCur], y: [yLo, yHi], mode: "lines",
        line: { dash: "dash", width: 2 }, name: "current g" },

      { x: solCur.ok ? [gCur, gCur] : [], y: solCur.ok ? [qCur, pCur] : [],
        mode: "markers",
        marker: { symbol: "x", size: 11, color: "green", line: { width: 2, color: "green" } },
        name: "current q,p" }
    ];

    const layout = {
      font: { size: 11 },
      margin: { l: 52, r: 10, t: 26, b: 38 },
      xaxis: { range: [0, gmax], title: "g (historic center location)" },
      yaxis: { range: [yLo, yHi], title: "location on the city line" },
      legend: {
        orientation: "h",
        yanchor: "bottom",
        y: 1.01,
        xanchor: "left",
        x: 0,
        font: { size: 10 }
      },
      height: elH("plot_fig1", 300)
    };

    Plotly.react("plot_fig1", data, layout, {displayModeBar: false, responsive: false});
  }

  function drawFig2() {
    const N = state.N, e = state.e, g = state.g;
    const tmin = 0.0;
    const tmax = 1.5;

    const n = 280;
    const ts = [];
    const qs = [];
    const ps = [];

    for (let i=0; i<n; i++) {
      const t = tmin + (tmax - tmin) * (i/(n-1));
      const sol = solveBounds(N, t, e, g);
      ts.push(t);
      qs.push(sol.ok ? sol.q : NaN);
      ps.push(sol.ok ? sol.p : NaN);
    }

    const solNow = solveBounds(N, state.t, e, g);
    const qNow = solNow.ok ? solNow.q : NaN;
    const pNow = solNow.ok ? solNow.p : NaN;

    const data = [
      { x: qs, y: ts, mode: "lines", name: "left edge q(t)" },
      { x: ps, y: ts, mode: "lines", name: "right edge p(t)" },

      { x: qs, y: ts, mode: "lines", line: { width: 0 }, showlegend: false },
      { x: ps, y: ts, mode: "lines", fill: "tonextx", opacity: 0.2, name: "business area" },

      { x: [g, g], y: [tmin, tmax], mode: "lines", line: { dash: "dash" }, name: "g (fixed)" },

      { x: [qNow, pNow], y: [state.t, state.t], mode: "markers", marker: { size: 8 }, name: "current t points" }
    ];

    const layout = {
      font: { size: 11 },
      margin: { l: 52, r: 10, t: 26, b: 38 },
      xaxis: { range: [-1.05, 1.05], title: "location on the city line" },
      yaxis: { range: [tmin, tmax], title: "t (transport cost)" },
      legend: {
        orientation: "h",
        yanchor: "bottom",
        y: 1.01,
        xanchor: "left",
        x: 0,
        font: { size: 10 }
      },
      height: elH("plot_fig2", 300)
    };

    Plotly.react("plot_fig2", data, layout, {displayModeBar: false, responsive: false});
  }

  function updateAll() {
    const sol = solveBounds(state.N, state.t, state.e, state.g);
    updateText(sol);
    drawLayout(sol);
    drawFig1();
    drawFig2();
  }

  function wireUp() {
    syncLabelsFromState();
    updateAll();

    ["N","e","g","t"].forEach((id) => {
      const el = document.getElementById(id);
      el.addEventListener("input", () => {
        state[id] = readSlider(id);
        syncLabelsFromState();
        updateAll();
      });
    });

    let rT = 0;
    window.addEventListener("resize", () => {
      clearTimeout(rT);
      rT = setTimeout(() => updateAll(), 120);
    });
  }

  wireUp();
</script>

</body>
</html>
"""
    out_path = Path(out_path)
    out_path.write_text(html, encoding="utf-8")
    return out_path

path = write_interactive_html("./data/yerevan_interactive/thesis_sectioned_dashboard.html")

try:
    display(IFrame(src=str(path), width=1400, height=860))
except Exception:
    display(HTML(path.read_text(encoding="utf-8")))

print(f"Wrote: {path.resolve()}")


Wrote: C:\Users\Nedric\Armenia\data\yerevan_interactive\thesis_sectioned_dashboard.html


In [81]:
# build_compare_business_areas_html.py
#
# Builds ONE HTML that:
# 1) loads ALL city business-area polygons from POLY_DIR/*.geojson (fixed polygons)
# 2) embeds your NEW Yerevan precomputed payload as gzip+base64 extracted from:
#    data/yerevan_interactive/yerevan_business_polygon_precomputed.html
# 3) renders both on a comparable METERS scale, centered at historic center (0,0)
# 4) compares shapes using ONLY the OUTER BOUNDARY (ignore holes; MultiPolygon -> largest exterior ring)
#
# Inputs expected:
# - data/city_centers/city_centers_business_polygon.csv
# - data/city_centers/business_area_polygons_geojson/*.geojson
# - data/yerevan_interactive/yerevan_business_polygon_precomputed.html   (gzip+b64 payload)
#
# Output:
# - data/compare_business_areas/compare_business_areas.html

import os
import re
import json
import math
import glob
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely import affinity

CITY_CSV = "data/city_centers/city_centers_business_polygon.csv"
POLY_DIR = "data/city_centers/business_area_polygons_geojson"
YEREVAN_PRECOMP_HTML = "data/yerevan_interactive/yerevan_business_polygon_precomputed.html"

OUT_DIR = "data/compare_business_areas"
OUT_HTML = os.path.join(OUT_DIR, "compare_business_areas.html")
os.makedirs(OUT_DIR, exist_ok=True)


def extract_payload_b64_from_yerevan_html(html_text: str) -> str:
    """
    Extracts:
      const PAYLOAD_GZ_B64 = "....";
    Returns the base64 string.
    """
    m = re.search(r'PAYLOAD_GZ_B64\s*=\s*"([^"]+)"\s*;', html_text)
    if not m:
        # some builds may use single quotes
        m = re.search(r"PAYLOAD_GZ_B64\s*=\s*'([^']+)'\s*;", html_text)
    if not m:
        raise RuntimeError("Could not find PAYLOAD_GZ_B64 in Yerevan HTML")
    return m.group(1)


def geom_to_meters_geojson_like(geom):
    if geom is None or geom.is_empty:
        return None

    if geom.geom_type == "Polygon":
        rings = []
        rings.append([[float(x), float(y)] for x, y in geom.exterior.coords])
        for interior in geom.interiors:
            rings.append([[float(x), float(y)] for x, y in interior.coords])
        return {"type": "Polygon", "coordinates": rings}

    if geom.geom_type == "MultiPolygon":
        polys = []
        for p in geom.geoms:
            rings = []
            rings.append([[float(x), float(y)] for x, y in p.exterior.coords])
            for interior in p.interiors:
                rings.append([[float(x), float(y)] for x, y in interior.coords])
            polys.append(rings)
        return {"type": "MultiPolygon", "coordinates": polys}

    return None


def load_csv_metadata(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    df["city"] = df["city"].astype(str).str.strip()
    return df


def safe_float(x):
    try:
        v = float(x)
        if not math.isfinite(v):
            return None
        return v
    except Exception:
        return None


def choose_utm_crs_for_polygon(poly_ll_gdf: gpd.GeoDataFrame, fallback_lonlat=None) -> str:
    try:
        crs = poly_ll_gdf.estimate_utm_crs()
        if crs is not None:
            return crs.to_string()
    except Exception:
        pass

    if fallback_lonlat:
        lon, lat = fallback_lonlat
        zone = int((lon + 180.0) // 6) + 1
        epsg = (32600 + zone) if (lat >= 0) else (32700 + zone)
        return f"EPSG:{epsg}"

    return "EPSG:3857"


def load_city_polygons_payload(csv_path: str, poly_dir: str):
    df = load_csv_metadata(csv_path)
    meta = {r["city"]: r for _, r in df.iterrows()}

    files = sorted(glob.glob(os.path.join(poly_dir, "*.geojson")))
    if not files:
        raise RuntimeError(f"No geojson polygons found in {poly_dir}")

    cities = {}
    max_abs = 0.0
    skipped = []

    for fp in files:
        try:
            gdf = gpd.read_file(fp)
        except Exception as e:
            skipped.append((os.path.basename(fp), f"read_error:{type(e).__name__}"))
            continue

        if gdf.empty or gdf.geometry.isna().all():
            skipped.append((os.path.basename(fp), "empty_geometry"))
            continue

        city = None
        for col in ["city", "name"]:
            if col in gdf.columns:
                v = str(gdf[col].iloc[0]).strip()
                if v and v.lower() != "nan":
                    city = v
                    break
        if not city:
            city = os.path.splitext(os.path.basename(fp))[0]

        # Yerevan is dynamic in this app
        if city.strip().lower() == "yerevan, armenia":
            continue

        geom_ll = gdf.geometry.iloc[0]
        if geom_ll is None or geom_ll.is_empty:
            skipped.append((city, "empty_geom"))
            continue

        row = meta.get(city, None)

        historic_lon = safe_float(row.get("historic_lon")) if row is not None else None
        historic_lat = safe_float(row.get("historic_lat")) if row is not None else None
        utm_crs = str(row.get("utm_crs")).strip() if row is not None else ""

        if historic_lon is None or historic_lat is None:
            skipped.append((city, "missing_historic_lonlat"))
            continue

        poly_ll_gdf = gpd.GeoDataFrame({"city": [city]}, geometry=[geom_ll], crs=gdf.crs or "EPSG:4326")
        poly_ll_gdf = poly_ll_gdf.to_crs("EPSG:4326")

        if (not utm_crs) or utm_crs.lower() == "nan":
            utm_crs = choose_utm_crs_for_polygon(poly_ll_gdf, fallback_lonlat=(historic_lon, historic_lat))

        try:
            geom_utm = poly_ll_gdf.to_crs(utm_crs).geometry.iloc[0]
            g_utm = gpd.GeoSeries([Point(historic_lon, historic_lat)], crs="EPSG:4326").to_crs(utm_crs).iloc[0]
        except Exception as e:
            skipped.append((city, f"project_error:{type(e).__name__}"))
            continue

        gx, gy = float(g_utm.x), float(g_utm.y)
        shifted = affinity.translate(geom_utm, xoff=-gx, yoff=-gy)

        gj = geom_to_meters_geojson_like(shifted)
        if gj is None:
            skipped.append((city, "unsupported_geom_type"))
            continue

        area_km2 = None
        if row is not None:
            area_km2 = safe_float(row.get("business_area_km2"))
        if area_km2 is None:
            area_km2 = float(shifted.area) / 1e6

        def update_max_abs(obj):
            nonlocal max_abs
            if obj["type"] == "Polygon":
                for ring in obj["coordinates"]:
                    for x, y in ring:
                        max_abs = max(max_abs, abs(float(x)), abs(float(y)))
            else:
                for poly in obj["coordinates"]:
                    for ring in poly:
                        for x, y in ring:
                            max_abs = max(max_abs, abs(float(x)), abs(float(y)))

        update_max_abs(gj)

        cities[city] = {
            "geom_m": gj,
            "area_km2": float(area_km2),
        }

    extent_m = float(max_abs) * 1.10 if max_abs > 0 else 5000.0

    print(f"Polygons found in folder: {len(files)}")
    print(f"Loaded cities: {len(cities)}")
    if skipped:
        print(f"Skipped: {len(skipped)}")
        for nm, reason in skipped[:80]:
            print("  -", nm, "=>", reason)

    return {"extent_m": extent_m, "cities": cities}


HTML_TEMPLATE = r"""<!doctype html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>Business areas comparison</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <style>
    :root { --border:#e6e6e6; --text:#111; --muted:#666; --bg:#fff; }
    body { margin:0; font-family: Arial, sans-serif; background:var(--bg); color:var(--text); }
    .wrap { max-width: 1200px; margin: 0 auto; padding: 28px 22px 18px; }
    h1 { margin: 0 0 10px; font-size: 44px; letter-spacing: -0.02em; }
    .desc { max-width: 980px; color: var(--muted); line-height: 1.35; font-size: 18px; margin-bottom: 22px; }

    .card {
      border-top: 1px solid var(--border);
      display: grid;
      grid-template-columns: 420px 1fr;
      gap: 0;
      min-height: 560px;
    }

    .left { padding: 20px 18px; border-right: 1px solid var(--border); }
    .sectionTitle { font-weight: 700; margin: 14px 0 10px; }
    .label { color: var(--text); margin-bottom: 8px; font-size: 14px; }
    .help { color: var(--muted); font-size: 12px; margin-top: 6px; }

    select{
      width: 100%;
      padding: 10px 12px;
      border: 1px solid var(--border);
      border-radius: 10px;
      font-size: 14px;
      outline: none;
      background: #fff;
    }

    .sliderBlock { margin-top: 20px; }
    .sliderName { font-weight: 700; margin: 18px 0 10px; }
    .sliderRow { display:flex; align-items:center; gap: 10px; }
    input[type="range"]{ width: 100%; }
    .valPill {
      min-width: 56px;
      text-align: right;
      font-weight: 700;
      font-variant-numeric: tabular-nums;
    }

    .right { position: relative; padding: 0; display:flex; flex-direction: column; }
    .plotWrap { position: relative; flex: 1 1 auto; min-height: 520px; background: #fff; }

    .plotLabel {
      position:absolute; color: #9a9a9a; font-size: 14px;
      user-select:none; pointer-events:none;
    }
    .plotLabel.top { top: 10px; left: 50%; transform: translateX(-50%); }
    .plotLabel.bottom { bottom: 10px; left: 50%; transform: translateX(-50%); }
    .plotLabel.left { left: 10px; top: 50%; transform: translateY(-50%); }
    .plotLabel.right { right: 10px; top: 50%; transform: translateY(-50%); }

    svg { width: 100%; height: 100%; display:block; }

    .footer { border-top: 1px solid var(--border); padding: 12px 18px; color: var(--muted); font-size: 14px; }

    .legendRow {
      display:flex; align-items:center; gap:14px; flex-wrap:wrap;
      padding: 12px 18px 6px;
    }
    .legendItem { display:flex; align-items:center; gap:8px; color: var(--muted); font-size: 13px; }
    .swatch { width: 14px; height: 10px; border-radius: 3px; border: 1px solid rgba(0,0,0,0.15); }

    .kpi {
      margin-top: 14px; padding-top: 14px;
      border-top: 1px solid var(--border);
      color: var(--muted);
      font-size: 13px;
      line-height: 1.35;
    }
    .kpi b { color: var(--text); }

    .halo {
      paint-order: stroke fill;
      stroke: rgba(255,255,255,0.95);
      stroke-width: 4;
      stroke-linejoin: round;
    }
  </style>
</head>

<body>
  <div class="wrap">
    <div class="desc">
      Сравнение бизнес-ареалов городов. Фигуры приведены к одной шкале и центрированы по историческому центру (0,0).
      Ереван меняется по слайдерам, берется из gzip+base64 предрасчета.
      Сходство считается только по внешнему контуру (без дыр, MultiPolygon сводится к крупнейшему контуру).
    </div>

    <div class="card">
      <div class="left">
        <div class="label">Выберите город, с которым хотите сравнить Ереван</div>
        <select id="citySelect"></select>

        <div class="sectionTitle" style="margin-top:18px;">Настройте показатели для Еревана</div>

        <div class="sliderBlock">
          <div class="sliderName">Transport factor</div>
          <div class="sliderRow">
            <input id="tSlider" type="range" min="0.10" max="2.00" step="0.01" value="1.00">
            <div class="valPill" id="tVal">1.00</div>
          </div>
          <div class="help" id="tSnap">snap: 1.00</div>

          <div class="sliderName" style="margin-top:18px;">Historic amenity strength</div>
          <div class="sliderRow">
            <input id="aSlider" type="range" min="0.10" max="2.00" step="0.01" value="1.00">
            <div class="valPill" id="aVal">1.00</div>
          </div>
          <div class="help" id="aSnap">snap: 1.00</div>

          <div class="kpi">
            <div><b>Ереван:</b> <span id="yKpi">...</span></div>
            <div><b>Город сравнения:</b> <span id="cKpi">...</span></div>
            <div><b>Самое похоже на:</b> <span id="matchCity">...</span></div>
          </div>
        </div>
      </div>

      <div class="right">
        <div class="legendRow">
          <div class="legendItem"><span class="swatch" style="background: rgba(0, 155, 190, 0.18);"></span>Yerevan business area</div>
          <div class="legendItem"><span class="swatch" style="background: rgba(240, 145, 110, 0.18);"></span><span id="legendCityName">...</span> business area</div>
          <div class="legendItem"><span class="swatch" style="background: rgba(220, 60, 60, 0.95); border:none;"></span>historical center of cities</div>
        </div>

        <div class="plotWrap">
          <div class="plotLabel top">выше</div>
          <div class="plotLabel bottom">ниже</div>
          <div class="plotLabel left">левее</div>
          <div class="plotLabel right">правее</div>

          <svg id="plot" preserveAspectRatio="xMidYMid meet">
            <line id="xAxis" x1="0" y1="0" x2="0" y2="0" stroke="#d7d7d7" stroke-width="1"></line>
            <line id="yAxis" x1="0" y1="0" x2="0" y2="0" stroke="#d7d7d7" stroke-width="1"></line>

            <path id="cityPoly" d="" fill="rgba(240, 145, 110, 0.18)" stroke="rgba(240, 145, 110, 0.95)" stroke-width="2" fill-rule="evenodd"></path>
            <path id="yerPoly" d="" fill="rgba(0, 155, 190, 0.18)" stroke="rgba(0, 155, 190, 0.95)" stroke-width="2" fill-rule="evenodd"></path>

            <circle id="originDot" r="6" cx="0" cy="0" fill="rgba(220, 60, 60, 0.95)"></circle>

            <text id="originLabel" class="halo" font-size="14" font-weight="700" fill="rgba(220, 60, 60, 0.95)">historical center of cities</text>
            <text id="cityLabel" class="halo" font-size="14" font-weight="700" fill="rgba(240, 145, 110, 0.95)"></text>
            <text id="yerLabel"  class="halo" font-size="14" font-weight="700" fill="rgba(0, 155, 190, 0.95)"></text>
          </svg>
        </div>

        <div class="footer">
          Сходство: радиальная сигнатура по внешнему контуру, инвариантная к повороту. Внутренние отверстия игнорируются.
        </div>
      </div>
    </div>
  </div>

  <script src="https://cdnjs.cloudflare.com/ajax/libs/proj4js/2.9.0/proj4.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/pako@2.1.0/dist/pako.min.js"></script>

  <script>
    const cityPayload = __CITY_PAYLOAD__;

    // embedded Yerevan gzip+base64 (copied from your Yerevan HTML)
    const YEREVAN_PAYLOAD_GZ_B64 = "__YEREVAN_PAYLOAD_GZ_B64__";

    function loadYerevanPayload() {
      const bin = Uint8Array.from(atob(YEREVAN_PAYLOAD_GZ_B64), c => c.charCodeAt(0));
      const jsonStr = pako.ungzip(bin, { to: "string" });
      return JSON.parse(jsonStr);
    }

    const yerevanData = loadYerevanPayload();

    proj4.defs("EPSG:32638", "+proj=utm +zone=38 +datum=WGS84 +units=m +no_defs");

    const cityNames = Object.keys(cityPayload.cities).sort();

    const citySelect = document.getElementById("citySelect");
    for (const nm of cityNames) {
      const opt = document.createElement("option");
      opt.value = nm;
      opt.textContent = nm;
      citySelect.appendChild(opt);
    }

    const defaultCity = cityNames.includes("Tokyo, Japan") ? "Tokyo, Japan" : (cityNames[0] || "");
    citySelect.value = defaultCity;

    const tSlider = document.getElementById("tSlider");
    const aSlider = document.getElementById("aSlider");
    const tValEl = document.getElementById("tVal");
    const aValEl = document.getElementById("aVal");
    const tSnapEl = document.getElementById("tSnap");
    const aSnapEl = document.getElementById("aSnap");
    const yKpiEl = document.getElementById("yKpi");
    const cKpiEl = document.getElementById("cKpi");
    const matchCityEl = document.getElementById("matchCity");
    const legendCityName = document.getElementById("legendCityName");

    const svg = document.getElementById("plot");
    const xAxis = document.getElementById("xAxis");
    const yAxis = document.getElementById("yAxis");

    const cityPolyEl = document.getElementById("cityPoly");
    const yerPolyEl = document.getElementById("yerPoly");

    const originDot = document.getElementById("originDot");
    const originLabel = document.getElementById("originLabel");
    const cityLabel = document.getElementById("cityLabel");
    const yerLabel = document.getElementById("yerLabel");

    const R = Math.max(500.0, cityPayload.extent_m);

    function svgSize() {
      const r = svg.getBoundingClientRect();
      return { w: Math.max(10, r.width), h: Math.max(10, r.height) };
    }

    function worldToSvg(x, y, w, h) {
      const s = Math.min(w, h) / (2.0 * R);
      const cx = w * 0.5;
      const cy = h * 0.5;
      return { x: cx + x * s, y: cy - y * s, s };
    }

    function ringToPathMeters(ring, w, h) {
      if (!ring || ring.length < 3) return "";
      let d = "";
      for (let i = 0; i < ring.length; i++) {
        const p = ring[i];
        const xy = worldToSvg(p[0], p[1], w, h);
        d += (i === 0 ? "M " : " L ") + xy.x.toFixed(2) + " " + xy.y.toFixed(2);
      }
      d += " Z";
      return d;
    }

    function geomMetersToPath(geom, w, h) {
      if (!geom) return "";
      let d = "";
      if (geom.type === "Polygon") {
        for (const ring of geom.coordinates) d += " " + ringToPathMeters(ring, w, h);
        return d.trim();
      }
      if (geom.type === "MultiPolygon") {
        for (const poly of geom.coordinates) {
          for (const ring of poly) d += " " + ringToPathMeters(ring, w, h);
        }
        return d.trim();
      }
      return "";
    }

    function geomCentroidMeters(geom) {
      if (!geom) return {x:0,y:0};
      let sx = 0, sy = 0, n = 0;

      function addRing(ring) {
        for (const p of ring) { sx += p[0]; sy += p[1]; n += 1; }
      }

      if (geom.type === "Polygon") addRing(geom.coordinates[0]);
      if (geom.type === "MultiPolygon") {
        for (const poly of geom.coordinates) addRing(poly[0]);
      }

      if (n === 0) return {x:0,y:0};
      return { x: sx / n, y: sy / n };
    }

    // Yerevan precomputed
    const tg = yerevanData.t_grid;
    const ag = yerevanData.a_grid;
    const g = yerevanData.g;         // only lon/lat in your new payload
    const precomp = yerevanData.precomp;

    // compute gX,gY in EPSG:32638 in browser
    const gXY = proj4("EPSG:4326", "EPSG:32638", [g.lon, g.lat]);
    const gX = gXY[0];
    const gY = gXY[1];

    tSlider.min = tg.min; tSlider.max = tg.max; tSlider.step = 0.01;
    aSlider.min = ag.min; aSlider.max = ag.max; aSlider.step = 0.01;

    function clamp(x, lo, hi) { return Math.max(lo, Math.min(hi, x)); }

    function snap(val, grid) {
      const v = clamp(val, grid.min, grid.max);
      const k = Math.round((v - grid.min) / grid.step);
      const snapped = grid.min + k * grid.step;
      return Number(snapped.toFixed(2));
    }

    function keyFor(t, a) {
      return `${t.toFixed(2)}_${a.toFixed(2)}`;
    }

    function lonLatToYerevanRelMeters(lon, lat) {
      const xy = proj4("EPSG:4326", "EPSG:32638", [lon, lat]);
      return [xy[0] - gX, xy[1] - gY];
    }

    function llGeomToRelMetersGeom(geom) {
      if (!geom) return null;

      if (geom.type === "Polygon") {
        const rings = geom.coordinates.map(ring =>
          ring.map(p => lonLatToYerevanRelMeters(p[0], p[1]))
        );
        return { type: "Polygon", coordinates: rings };
      }

      if (geom.type === "MultiPolygon") {
        const polys = geom.coordinates.map(poly =>
          poly.map(ring => ring.map(p => lonLatToYerevanRelMeters(p[0], p[1])))
        );
        return { type: "MultiPolygon", coordinates: polys };
      }

      return null;
    }

    // Label collision avoidance (approx)
    function textBoxApprox(text, x, y, fontSize) {
      const w = Math.max(10, text.length * fontSize * 0.62);
      const h = fontSize * 1.25;
      return { x0: x, y0: y - h, x1: x + w, y1: y };
    }

    function boxesOverlap(a, b) {
      return !(a.x1 < b.x0 || a.x0 > b.x1 || a.y1 < b.y0 || a.y0 > b.y1);
    }

    function placeLabel(el, text, wx, wy, w, h, usedBoxes, preferredOffsets) {
      el.textContent = text;

      const base = worldToSvg(wx, wy, w, h);
      const fontSize = 14;

      const offsets = preferredOffsets || [
        [12, -12], [12, 18], [-160, -12], [-160, 18], [12, -30], [12, 34], [-220, -30], [-220, 34]
      ];

      for (const off of offsets) {
        const px = base.x + off[0];
        const py = base.y + off[1];
        const box = textBoxApprox(text, px, py, fontSize);

        let ok = true;
        for (const ub of usedBoxes) {
          if (boxesOverlap(box, ub)) { ok = false; break; }
        }
        if (ok) {
          el.setAttribute("x", px.toFixed(2));
          el.setAttribute("y", py.toFixed(2));
          usedBoxes.push(box);
          return;
        }
      }

      const px = base.x + 12;
      const py = base.y - 12;
      el.setAttribute("x", px.toFixed(2));
      el.setAttribute("y", py.toFixed(2));
      usedBoxes.push(textBoxApprox(text, px, py, fontSize));
    }

    function updateAxes(w, h) {
      const o = worldToSvg(0, 0, w, h);
      xAxis.setAttribute("x1", 0);
      xAxis.setAttribute("y1", o.y);
      xAxis.setAttribute("x2", w);
      xAxis.setAttribute("y2", o.y);

      yAxis.setAttribute("x1", o.x);
      yAxis.setAttribute("y1", 0);
      yAxis.setAttribute("x2", o.x);
      yAxis.setAttribute("y2", h);

      originDot.setAttribute("cx", o.x);
      originDot.setAttribute("cy", o.y);
    }

    // Similarity: radial signature computed ONLY from OUTER BOUNDARY
    const K_SIG = 72;
    const MAX_PTS = 2500;

    function ringAreaAbs(ring) {
      let a = 0;
      for (let i = 0; i < ring.length - 1; i++) {
        const x0 = ring[i][0], y0 = ring[i][1];
        const x1 = ring[i+1][0], y1 = ring[i+1][1];
        a += x0 * y1 - x1 * y0;
      }
      return Math.abs(0.5 * a);
    }

    function ringCentroidSigned(ring) {
      let a = 0, cx2 = 0, cy2 = 0;
      for (let i = 0; i < ring.length - 1; i++) {
        const x0 = ring[i][0], y0 = ring[i][1];
        const x1 = ring[i+1][0], y1 = ring[i+1][1];
        const cross = x0 * y1 - x1 * y0;
        a += cross;
        cx2 += (x0 + x1) * cross;
        cy2 += (y0 + y1) * cross;
      }
      a *= 0.5;
      if (Math.abs(a) < 1e-9) return { x: ring[0][0], y: ring[0][1], a: 0 };
      return { x: cx2 / (6 * a), y: cy2 / (6 * a), a: a };
    }

    function largestOuterRingOnly(geom) {
      if (!geom) return null;

      let bestRing = null;
      let bestArea = -1;

      if (geom.type === "Polygon") {
        const ring = geom.coordinates[0];
        return ring || null;
      }

      if (geom.type === "MultiPolygon") {
        for (const poly of geom.coordinates) {
          const ring = poly && poly[0] ? poly[0] : null;
          if (!ring) continue;
          const a = ringAreaAbs(ring);
          if (a > bestArea) {
            bestArea = a;
            bestRing = ring;
          }
        }
        return bestRing;
      }

      return null;
    }

    function fillZerosCircular(arr) {
      const n = arr.length;
      const out = arr.slice();
      const nz = [];
      for (let i = 0; i < n; i++) if (out[i] > 0) nz.push(i);
      if (nz.length === 0) return out;

      for (let i = 0; i < n; i++) {
        if (out[i] > 0) continue;
        let best = nz[0], bestd = 1e9;
        for (const j of nz) {
          const d = Math.min((i - j + n) % n, (j - i + n) % n);
          if (d < bestd) { bestd = d; best = j; }
        }
        out[i] = out[best];
      }
      return out;
    }

    function normalizeL2(v) {
      let s = 0;
      for (let i = 0; i < v.length; i++) s += v[i] * v[i];
      const n = Math.sqrt(s);
      if (n < 1e-12) return v.slice();
      return v.map(x => x / n);
    }

    function computeSignatureOuterOnly(geom, K) {
      const ring = largestOuterRingOnly(geom);
      if (!ring || ring.length < 4) return { sig: new Array(K).fill(0), area: 0 };

      const rc = ringCentroidSigned(ring);
      const cx0 = rc.x, cy0 = rc.y;
      const areaAbs = Math.abs(rc.a);

      const scale = Math.sqrt(Math.max(areaAbs, 1e-9));
      const bins = new Array(K).fill(0);

      const step = Math.max(1, Math.floor(ring.length / MAX_PTS));
      for (let i = 0; i < ring.length; i += step) {
        const dx = ring[i][0] - cx0;
        const dy = ring[i][1] - cy0;
        const r = Math.sqrt(dx*dx + dy*dy);

        let ang = Math.atan2(dy, dx);
        let t = (ang + Math.PI) / (2 * Math.PI);
        let k = Math.floor(t * K);
        if (k >= K) k = K - 1;

        if (r > bins[k]) bins[k] = r;
      }

      const filled = fillZerosCircular(bins).map(x => x / scale);
      return { sig: normalizeL2(filled), area: areaAbs / 1e6 };
    }

    function minCyclicMSE(a, b) {
      const n = a.length;
      let best = Infinity;
      for (let shift = 0; shift < n; shift++) {
        let s = 0;
        for (let i = 0; i < n; i++) {
          const d = a[i] - b[(i + shift) % n];
          s += d * d;
        }
        const mse = s / n;
        if (mse < best) best = mse;
      }
      return best;
    }

    // Precompute city signatures once
    const citySigIndex = {};
    for (const nm of cityNames) {
      const g0 = cityPayload.cities[nm].geom_m;
      citySigIndex[nm] = computeSignatureOuterOnly(g0, K_SIG);
    }

    let pending = null;
    function scheduleUpdate() {
      if (pending) clearTimeout(pending);
      pending = setTimeout(update, 80);
    }

    citySelect.addEventListener("change", scheduleUpdate);
    tSlider.addEventListener("input", scheduleUpdate);
    aSlider.addEventListener("input", scheduleUpdate);
    window.addEventListener("resize", scheduleUpdate);

    function update() {
      const cityName = citySelect.value;
      legendCityName.textContent = cityName;

      const { w, h } = svgSize();
      updateAxes(w, h);

      // comparison city polygon
      const cityObj = cityPayload.cities[cityName];
      const cityGeom = cityObj ? cityObj.geom_m : null;
      cityPolyEl.setAttribute("d", geomMetersToPath(cityGeom, w, h));

      // slider raw
      const tRaw = parseFloat(tSlider.value);
      const aRaw = parseFloat(aSlider.value);
      tValEl.textContent = tRaw.toFixed(2);
      aValEl.textContent = aRaw.toFixed(2);

      // snap
      const tS = snap(tRaw, tg);
      const aS = snap(aRaw, ag);
      tSnapEl.textContent = "snap: " + tS.toFixed(2);
      aSnapEl.textContent = "snap: " + aS.toFixed(2);

      // pick precomputed entry
      const entry = precomp[keyFor(tS, aS)];

      let yerGeomRel = null;
      if (entry && entry.poly_geom) {
        yerGeomRel = llGeomToRelMetersGeom(entry.poly_geom);
      }

      yerPolyEl.setAttribute("d", geomMetersToPath(yerGeomRel, w, h));

      if (entry) {
        yKpiEl.textContent = entry.area_km2.toFixed(2) + " km2 | sep: " + entry.sep_m.toFixed(0) + " m";
      } else {
        yKpiEl.textContent = "no data";
      }

      if (cityObj) cKpiEl.textContent = cityObj.area_km2.toFixed(2) + " km2";
      else cKpiEl.textContent = "...";

      // Labels with collision avoidance
      const used = [];
      placeLabel(originLabel, "historical center of cities", 0, 0, w, h, used, [[12, -14], [12, -34], [12, 22]]);

      if (cityGeom) {
        const cc = geomCentroidMeters(cityGeom);
        placeLabel(cityLabel, cityName + " business area", cc.x, cc.y, w, h, used, [[12, -12], [12, 18], [-220, -12], [-220, 18]]);
      } else {
        cityLabel.textContent = "";
      }

      if (yerGeomRel) {
        const yc = geomCentroidMeters(yerGeomRel);
        placeLabel(yerLabel, "Yerevan business area", yc.x, yc.y, w, h, used, [[12, 18], [12, -12], [-200, 18], [-200, -12]]);
      } else {
        yerLabel.textContent = "";
      }

      // Similarity match (outer bounds only)
      if (yerGeomRel && cityNames.length > 0) {
        const yd = computeSignatureOuterOnly(yerGeomRel, K_SIG);

        let bestCity = null;
        let bestScore = Infinity;

        // set to 0 for pure outer-shape only
        const W_AREA = 0.15;

        for (const nm of cityNames) {
          const cd = citySigIndex[nm];
          if (!cd || !cd.sig) continue;

          const shapeScore = minCyclicMSE(yd.sig, cd.sig);

          let areaScore = 0;
          if (W_AREA > 0 && yd.area > 0 && cd.area > 0) {
            areaScore = Math.abs(Math.log(yd.area / cd.area));
          }

          const score = shapeScore + W_AREA * areaScore;
          if (score < bestScore) {
            bestScore = score;
            bestCity = nm;
          }
        }

        if (bestCity) matchCityEl.textContent = bestCity + " (score " + bestScore.toFixed(4) + ")";
        else matchCityEl.textContent = "...";
      } else {
        matchCityEl.textContent = "...";
      }
    }

    update();
  </script>
</body>
</html>
"""


def main():
    if not os.path.exists(CITY_CSV):
        raise SystemExit(f"Missing: {CITY_CSV}")
    if not os.path.exists(YEREVAN_PRECOMP_HTML):
        raise SystemExit(f"Missing: {YEREVAN_PRECOMP_HTML}")

    city_payload = load_city_polygons_payload(CITY_CSV, POLY_DIR)
    city_payload_js = json.dumps(city_payload, ensure_ascii=False)

    with open(YEREVAN_PRECOMP_HTML, "r", encoding="utf-8") as f:
        ytxt = f.read()
    y_b64 = extract_payload_b64_from_yerevan_html(ytxt)

    html = HTML_TEMPLATE.replace("__CITY_PAYLOAD__", city_payload_js)
    html = html.replace("__YEREVAN_PAYLOAD_GZ_B64__", y_b64)

    with open(OUT_HTML, "w", encoding="utf-8") as f:
        f.write(html)

    print("Saved:", OUT_HTML)


if __name__ == "__main__":
    main()


Polygons found in folder: 91
Loaded cities: 90
Saved: data/compare_business_areas\compare_business_areas.html


In [87]:
import os
import html as _html
import urllib.parse
from string import Template
from pathlib import Path
import shutil


# =========================
# Small helpers
# =========================
def _escape(s: str) -> str:
    return _html.escape(str(s), quote=True)


def svg_placeholder_data_uri(label: str, w: int = 1600, h: int = 1000) -> str:
    safe = _html.escape(label)
    svg = f"""<svg xmlns="http://www.w3.org/2000/svg" width="{w}" height="{h}" viewBox="0 0 {w} {h}">
  <defs>
    <linearGradient id="g" x1="0" y1="0" x2="1" y2="1">
      <stop offset="0" stop-color="#f2f2f2"/>
      <stop offset="1" stop-color="#dcdcdc"/>
    </linearGradient>
  </defs>
  <rect width="{w}" height="{h}" rx="48" fill="url(#g)"/>
  <rect x="48" y="48" width="{w-96}" height="{h-96}" rx="36" fill="none" stroke="#bdbdbd" stroke-width="4"/>
  <text x="50%" y="52%" text-anchor="middle" font-family="Inter, Arial, sans-serif" font-size="54" fill="#333">{safe}</text>
  <text x="50%" y="59%" text-anchor="middle" font-family="Inter, Arial, sans-serif" font-size="24" fill="#666">
    Replace with assets/ image
  </text>
</svg>"""
    return "data:image/svg+xml;charset=utf-8," + urllib.parse.quote(svg)


def _clone_step(base: dict, **overrides) -> dict:
    out = dict(base)
    out.update(overrides)
    return out


# =========================
# Config
# =========================
# Idle scroll "slow-down" around first and last scenario
IDLE_LEAD_STEPS = 3          # number of invisible spacer steps before the first real scenario card
IDLE_TAIL_STEPS = 2          # number of invisible spacer steps after the last real scenario card
IDLE_LEAD_PAD_VH = 22        # vertical padding per idle step (vh)
IDLE_TAIL_PAD_VH = 22        # vertical padding per idle step (vh)
OUT_DIR = "data/yerevan_interactive"


def add_idle_spacer_steps(
    scenario_steps: list[dict],
    lead_count: int = 1,
    tail_count: int = 1,
    lead_pad_vh: int = 22,
    tail_pad_vh: int = 22,
) -> list[dict]:
    """
    Adds invisible "idle" scroll steps that keep the same scenario values, so the sticky viz
    stays pinned longer and the minibar/progress bar has time to be seen on the first and last scenario.
    """
    if not scenario_steps:
        return scenario_steps

    first = scenario_steps[0]
    last = scenario_steps[-1]

    out: list[dict] = []

    # Lead idle steps (use first scenario values, keep same progress/title)
    for _ in range(max(0, int(lead_count))):
        out.append(
            _clone_step(
                first,
                idle=True,
                pad_vh=int(lead_pad_vh),
                heading="",
                body="",
                hint="",
            )
        )

    out.extend(scenario_steps)

    # Tail idle steps (use last scenario values, keep same progress/title)
    for _ in range(max(0, int(tail_count))):
        out.append(
            _clone_step(
                last,
                idle=True,
                pad_vh=int(tail_pad_vh),
                heading="",
                body="",
                hint="",
            )
        )

    return out


def assign_progress_percent(scenario_steps: list[dict]) -> list[dict]:
    """
    Assign a stable progress percent per scenario (0..100), then idle steps can copy it.
    """
    if not scenario_steps:
        return scenario_steps

    n = len(scenario_steps)
    if n == 1:
        return [_clone_step(scenario_steps[0], prog=100.0)]

    out: list[dict] = []
    for i, st in enumerate(scenario_steps):
        pct = (i / (n - 1)) * 100.0
        out.append(_clone_step(st, prog=float(pct)))
    return out


# =========================
# Compare section: embed + autosize (landing) and height reporter (compare page)
# =========================
def patch_landing_add_compare_embed_css(landing_html: str) -> str:
    css = r"""

/* ---- injected: compare section full width + iframe embed ---- */
#compare{
  scroll-margin-top: calc(var(--navH) + 16px) !important;
}

#compare .container{
  max-width: none !important;
  width: 100% !important;
  box-sizing: border-box !important;
  padding-left: var(--modelPad, 12px) !important;
  padding-right: var(--modelPad, 12px) !important;
  margin-left: auto !important;
  margin-right: auto !important;
}

/* Borderless embed wrapper */
#compare .embedCard{
  background: transparent !important;   /* or #fff if you prefer */
  border: 0 !important;
  box-shadow: none !important;
  border-radius: var(--radius2) !important;
  overflow: hidden !important;
}

#compare .embedCard iframe{
  width: 100%;
  height: 700px;        /* initial, JS will override */
  border: 0;
  display: block;
  overflow: hidden;
}
/* ---- end injected block ---- */
"""
    style_close = landing_html.rfind("</style>")
    if style_close == -1:
        raise ValueError("Could not find </style> to inject compare embed CSS.")
    return landing_html[:style_close] + css + "\n" + landing_html[style_close:]


def patch_landing_insert_compare_before_explain(
    landing_html: str,
    compare_href: str = "compare_business_areas.html",
    title: str = "Business areas comparison",
    sub: str = "Compare Yerevan’s dynamic business area with other cities on a common meters scale.",
) -> str:
    """
    Inserts the compare section immediately before the Explanation section.
    No button.
    """
    block = f"""
  <section class="section" id="compare">
    <div class="container">
      <div class="sectionTitle">
        <h2>{_escape(title)}</h2>
        <p>{_escape(sub)}</p>
      </div>

      <div class="embedCard">
        <iframe id="compareFrame" src="{_escape(compare_href)}" title="{_escape(title)}" loading="eager" scrolling="no"></iframe>
      </div>
    </div>
  </section>
"""
    anchor = '<section class="section" id="explain">'
    if anchor not in landing_html:
        raise ValueError("Could not find the Explanation section anchor.")
    return landing_html.replace(anchor, block + "\n" + anchor, 1)


def patch_landing_add_compare_autosize_js(landing_html: str) -> str:
    """
    Autosize compare iframe:
    - Primary: accept postMessage({type:"compareHeight", height:<px>}) from compare page
    - Also sends postMessage({type:"requestHeight"}) to request updates
    - Same-origin fallback reads .wrap height (not document scrollHeight) to avoid growth loops
    """
    if "compareIframeAutosize" in landing_html:
        return landing_html

    injected = r"""
    // ---- injected: compare iframe autosize (stable, no growth loop) ----
    (function compareIframeAutosize() {
      const compareFrame = document.getElementById("compareFrame");
      if (!compareFrame) return;

      const _COMPARE_PAD = 24; // small safety pad, avoid runaway growth
      let _lastSet = 0;

      function setCompareHeight(px) {
        if (!compareFrame) return;
        if (typeof px !== "number" || !isFinite(px) || px <= 0) return;

        const next = Math.ceil(px + _COMPARE_PAD);

        // Prevent tiny oscillations
        if (_lastSet && Math.abs(next - _lastSet) <= 2) return;

        _lastSet = next;
        compareFrame.style.height = next + "px";
      }

      // Receive height from compare page
      window.addEventListener("message", (event) => {
        const d = event && event.data;
        if (!d || typeof d !== "object") return;
        if (d.type === "compareHeight" && typeof d.height === "number") {
          setCompareHeight(d.height);
        }
      });

      function requestCompareHeight() {
        try {
          if (compareFrame && compareFrame.contentWindow) {
            compareFrame.contentWindow.postMessage({ type: "requestHeight" }, "*");
          }
        } catch (e) {}
      }

      // Same-origin fallback: measure .wrap (content), not documentElement.scrollHeight
      function _compareDoc() {
        try {
          return compareFrame.contentDocument || (compareFrame.contentWindow && compareFrame.contentWindow.document) || null;
        } catch (e) {
          return null;
        }
      }

      function _computeContentHeightSameOrigin(doc) {
        try {
          const wrap = doc && doc.querySelector && doc.querySelector(".wrap");
          if (wrap) {
            const h = Math.max(wrap.scrollHeight || 0, wrap.offsetHeight || 0);
            return (h && isFinite(h)) ? Math.ceil(h) : 0;
          }
        } catch (e) {}
        return 0;
      }

      function resizeCompareFrameSameOrigin() {
        const doc = _compareDoc();
        if (!doc) return;
        const h = _computeContentHeightSameOrigin(doc);
        if (h) setCompareHeight(h);
      }

      compareFrame.setAttribute("scrolling", "no");
      compareFrame.style.overflow = "hidden";

      compareFrame.addEventListener("load", () => {
        requestCompareHeight();
        setTimeout(requestCompareHeight, 120);
        setTimeout(requestCompareHeight, 350);
        setTimeout(requestCompareHeight, 900);

        resizeCompareFrameSameOrigin();
        setTimeout(resizeCompareFrameSameOrigin, 120);
        setTimeout(resizeCompareFrameSameOrigin, 900);
      });

      window.addEventListener("resize", () => {
        requestCompareHeight();
        resizeCompareFrameSameOrigin();
        setTimeout(() => {
          requestCompareHeight();
          resizeCompareFrameSameOrigin();
        }, 250);
      });
    })();
    // ---- end injected block ----
"""

    needle = "\n    // Init\n"
    if needle in landing_html:
        return landing_html.replace(needle, "\n" + injected + needle, 1)

    idx = landing_html.rfind("</script>")
    if idx == -1:
        raise ValueError("Could not find </script> to inject compare autosize JS.")
    return landing_html[:idx] + "\n" + injected + "\n" + landing_html[idx:]


def patch_compare_add_height_postmessage(compare_html: str) -> str:
    """
    Injects postMessage height reporter into compare_business_areas.html.
    IMPORTANT: avoids infinite-growth loop by measuring .wrap height (content height),
    not documentElement.scrollHeight (which tracks iframe viewport).
    """
    if "compareHeightReporter" in compare_html:
        return compare_html

    injected = r"""
  <script id="compareHeightReporter">
  (function() {
    function _contentHeight() {
      try {
        var wrap = document.querySelector(".wrap");
        if (wrap) {
          var h = Math.max(wrap.scrollHeight || 0, wrap.offsetHeight || 0);
          if (h && isFinite(h)) return Math.ceil(h);
        }

        // Fallback only if .wrap not found
        var b = document.body;
        if (b) {
          var hb = Math.max(b.scrollHeight || 0, b.offsetHeight || 0);
          if (hb && isFinite(hb)) return Math.ceil(hb);
        }
      } catch (e) {}
      return 0;
    }

    function _postHeight() {
      try {
        var h = _contentHeight();
        if (!h || !isFinite(h) || h < 50) return;
        if (window.parent) {
          window.parent.postMessage({ type: "compareHeight", height: h }, "*");
        }
      } catch (e) {}
    }

    function _burst() {
      _postHeight();
      setTimeout(_postHeight, 60);
      setTimeout(_postHeight, 180);
      setTimeout(_postHeight, 420);
      setTimeout(_postHeight, 900);
    }

    window.addEventListener("message", function(event) {
      var d = event && event.data;
      if (!d || typeof d !== "object") return;
      if (d.type === "requestHeight") _burst();
    });

    window.addEventListener("load", _burst);
    window.addEventListener("resize", function() {
      _postHeight();
      setTimeout(_postHeight, 120);
    });

    if ("ResizeObserver" in window) {
      try {
        var ro = new ResizeObserver(function() { _postHeight(); });
        var wrap = document.querySelector(".wrap");
        if (wrap) ro.observe(wrap);
      } catch (e) {}
    }

    // Wrap update() so height is re-sent after redraws
    try {
      if (typeof update === "function" && !update.__heightWrapped) {
        var _u = update;
        var wrapped = function() {
          var r = _u.apply(this, arguments);
          _burst();
          return r;
        };
        wrapped.__heightWrapped = true;
        update = wrapped;
      }
    } catch (e) {}

    _burst();
  })();
  </script>
"""

    idx = compare_html.rfind("</body>")
    if idx == -1:
        idx = compare_html.rfind("</html>")
    if idx == -1:
        raise ValueError("Could not find </body> or </html> to inject height reporter.")
    return compare_html[:idx] + injected + "\n" + compare_html[idx:]


# =========================
# Patch: landing CSS layout tweaks for model section
# =========================
def patch_landing_for_model_focus_zoom(landing_html: str) -> str:
    css_inject = r"""

/* ---- injected: make model section full width + compact steps + no minibar overlap ---- */

:root{
  --navH: 68px;     /* JS overwrites */
  --miniBarH: 0px;  /* becomes >0 only while pinned */
  --modelPad: 12px; /* side padding for model section */
  --gapTop: 10px;
  --stepsW: 360px;  /* compact annotation width */
}

/* Top menu always visible */
.nav{
  position: fixed !important;
  top: 0 !important;
  left: 0 !important;
  right: 0 !important;
  z-index: 300 !important;
}

/* Page starts below fixed nav */
body{
  padding-top: var(--navH) !important;
}

/* Keep anchor jumps from hiding under nav */
#gallery1, #model, #gallery2, #explain{
  scroll-margin-top: calc(var(--navH) + 16px) !important;
}

/* Make only the model section nearly full width with small margins */
#model .container{
  max-width: none !important;
  width: 100% !important;
  box-sizing: border-box !important;
  padding-left: var(--modelPad, 12px) !important;
  padding-right: var(--modelPad, 12px) !important;
  margin-left: auto !important;
  margin-right: auto !important;
}

/* The header inside model should also align with new width */
#model .modelHeader{
  padding-left: 0 !important;
  padding-right: 0 !important;
}

/* Force the model grid to: compact left, huge right */
#model .modelGrid{
  display: grid !important;
  grid-template-columns: var(--stepsW) 1fr !important;
  gap: 18px !important;
  align-items: start !important;
}

/* Ensure the right column can expand fully */
#model .stickyViz{
  min-width: 0 !important;
}

/* Compact the steps column */
#model .stepsCol{
  width: var(--stepsW) !important;
  max-width: var(--stepsW) !important;
  padding-right: 0 !important;
  opacity: 0.98 !important;
}

/* Make step cards compact */
#model .stepCard{
  max-width: var(--stepsW) !important;
  padding: 12px 12px 10px 12px !important;
  border-radius: 20px !important;
}
#model .stepKicker{
  font-size: 10px !important;
}
#model .stepCard h3{
  font-size: 16px !important;
  margin-bottom: 6px !important;
}
#model .stepBody{
  font-size: 13px !important;
  line-height: 1.45 !important;
}
#model .chip{
  font-size: 11px !important;
  padding: 5px 9px !important;
}

/* Sticky map: always stick below nav, plus minibar when pinned */
#model .stickyViz{
  position: sticky !important;
  top: calc(var(--navH) + var(--miniBarH) + var(--gapTop)) !important;
  height: calc(100vh - var(--navH) - var(--miniBarH) - (2 * var(--gapTop))) !important;
  transition: none !important;
}

/* Hide minibar by default */
#model .modelMiniBar{
  display: none !important;
}

/* Show minibar only while pinned, below nav */
body.modelPinned{
  --miniBarH: 56px;
}

body.modelPinned #model .modelMiniBar{
  display: flex !important;
  position: fixed !important;
  top: var(--navH) !important;
  left: 0 !important;
  right: 0 !important;
  z-index: 250 !important;
  background: rgba(250,250,250,.94) !important;
  backdrop-filter: blur(10px) !important;
  border-bottom: 1px solid rgba(15,15,16,.12) !important;
  padding: 8px 18px 12px 18px !important;
}

/* Extra safety: when pinned, keep stickyViz pushed down */
body.modelPinned #model .stickyViz{
  top: calc(var(--navH) + var(--miniBarH) + var(--gapTop)) !important;
  height: calc(100vh - var(--navH) - var(--miniBarH) - (2 * var(--gapTop))) !important;
}

/* Smaller overall section paddings can help the map feel bigger */
#model.modelWrap, .modelWrap{
  padding-left: 0 !important;
  padding-right: 0 !important;
}

/* Mobile */
@media (max-width: 980px){
  :root{
    --stepsW: 100%;
    --modelPad: 12px;
  }

  #model .modelGrid{
    grid-template-columns: 1fr !important;
  }

  #model .stickyViz{
    position: relative !important;
    top: 0 !important;
    height: 70vh !important;
  }

  body.modelPinned{
    --miniBarH: 54px;
  }
}

/* ---- end injected block ---- */
"""

    style_close = landing_html.rfind("</style>")
    if style_close == -1:
        raise ValueError("Could not find </style> to inject CSS.")
    return landing_html[:style_close] + css_inject + "\n" + landing_html[style_close:]


# =========================
# Patch: center map zoom in viz
# =========================
def patch_center_map_button_zoom(viz_html: str, center_zoom: float = 15.0) -> str:
    import re

    def sub_first(pattern: str, html: str) -> tuple[str, int]:
        rx = re.compile(pattern, re.DOTALL)

        def _repl(m: re.Match) -> str:
            return m.group(1) + str(center_zoom) + m.group(3)

        return rx.subn(_repl, html, count=1)

    # 1) Patch the Center button handler, if it exists
    viz_html, n = sub_first(
        r"(map\.setView\(\s*\[\s*g\.lat\s*,\s*g\.lon\s*\]\s*,\s*)(\d+(?:\.\d+)?)(\s*,)",
        viz_html
    )
    if n:
        return viz_html

    # 2) Otherwise patch the Leaflet init setView (L.map(...).setView(...))
    viz_html, _ = sub_first(
        r"(\.setView\(\s*\[\s*g\.lat\s*,\s*g\.lon\s*\]\s*,\s*)(\d+(?:\.\d+)?)(\s*\))",
        viz_html
    )
    return viz_html


# =========================
# Patch 1: scrolly control hooks for iframe (viz)
# =========================
def patch_interactive_for_scrolly(interactive_html: str) -> str:
    injection = r"""
    // ---- scrollytelling control hooks (injected) ----
    function _dispatchInput(el) {
      try {
        el.dispatchEvent(new Event("input", { bubbles: true }));
        el.dispatchEvent(new Event("change", { bubbles: true }));
      } catch (e) {}
    }

    function setSliders(tVal, aVal) {
      const tSlider = document.getElementById("tSlider");
      const aSlider = document.getElementById("aSlider");
      if (!tSlider || !aSlider) return;

      if (typeof tVal === "number" && isFinite(tVal)) tSlider.value = String(tVal);
      if (typeof aVal === "number" && isFinite(aVal)) aSlider.value = String(aVal);

      _dispatchInput(tSlider);
      _dispatchInput(aSlider);

      try {
        if (typeof update === "function") update();
      } catch (e) {}
    }

    function setMapView(lat, lon, zoom) {
      if (typeof lat === "number" && typeof lon === "number" && typeof zoom === "number") {
        try {
          map.setView([lat, lon], zoom, { animate: true });
        } catch (e) {}
      }
    }

    let _invT = 0;
    function invalidateMapSize() {
      try {
        clearTimeout(_invT);
        _invT = setTimeout(() => map.invalidateSize(), 0);
      } catch (e) {}
    }

    function postReady() {
      try {
        window.parent && window.parent.postMessage({ type: "vizReady" }, "*");
      } catch (e) {}
    }

    window.addEventListener("message", (event) => {
      const msg = event.data;
      if (!msg || typeof msg !== "object") return;

      if (msg.type === "setSliders") setSliders(msg.t, msg.a);
      if (msg.type === "setView") setMapView(msg.lat, msg.lon, msg.zoom);
      if (msg.type === "invalidateSize") invalidateMapSize();

      if (msg.type === "ping") postReady();
    });

    postReady();
    // ---- end injected block ----
    """

    anchor = "    // Initial draw"
    if anchor in interactive_html:
        return interactive_html.replace(anchor, injection + "\n" + anchor, 1)

    idx = interactive_html.rfind("</script>")
    if idx == -1:
        raise ValueError("Could not find </script> to inject scrolly hooks.")
    return interactive_html[:idx] + injection + "\n" + interactive_html[idx:]


# =========================
# Patch 2: iframe UI layout (shrink side panels so the map is bigger)
# =========================
def patch_interactive_ui_left_right_vertical_sliders(html_str: str) -> str:
    import re

    old_body_pattern = re.compile(
        r'<div id="map"></div>\s*<div id="controls">.*?</div>\s*',
        re.DOTALL
    )

    # Body: keep only the top labels in panels
    new_body = r"""
  <div id="app">
    <div id="topRow">

      <div class="panel" id="leftPanel">
        <div class="panelKicker">Transport</div>

        <div class="panelSliderWrap">
          <div class="sliderStack" id="tStack">
            <div class="vTicks" id="tTicks" aria-hidden="true"></div>
            <div class="thumbLabel" id="tVal"></div>
            <input id="tSlider" class="vSlider pretty" type="range"
                   min="0" max="2.00" step="0.01" value="1.00"
                   aria-label="Transport slider">
          </div>
        </div>
      </div>

      <div class="mapPanel" id="mapPanel">
        <div id="map"></div>
      </div>

      <div class="panel" id="rightPanel">
        <div class="panelKicker">Amenities</div>

        <div class="panelSliderWrap">
          <div class="sliderStack" id="aStack">
            <div class="vTicks" id="aTicks" aria-hidden="true"></div>
            <div class="thumbLabel" id="aVal"></div>
            <input id="aSlider" class="vSlider pretty" type="range"
                   min="0" max="2.00" step="0.01" value="1.00"
                   aria-label="Amenities slider">
          </div>
        </div>
      </div>

    </div>

    <div id="bottomBar">
      <div class="barLeft">
        <div class="barLabel">Distance</div>
        <button id="centerBtn" class="barBtn" type="button">Center map</button>
      </div>
      <div class="barValue" id="muInfo"></div>
    </div>
  </div>
"""
    if not old_body_pattern.search(html_str):
        raise ValueError("Could not find the expected map+controls block to replace.")
    html_str = old_body_pattern.sub(new_body, html_str, count=1)

    style_pattern = re.compile(r"<style>.*?</style>", re.DOTALL)

    # Style: make side panels much thinner + slim slider UI
    new_style = r"""
  <style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700;800;900&display=swap');

    html, body { height: 100%; }
    body { margin: 0; font-family: Inter, Arial, sans-serif; overflow: hidden; background: #fff; }

    #app { height: 100%; display: flex; flex-direction: column; }

    /* Thinner side panels */
    #topRow {
      flex: 1;
      min-height: 0;
      display: grid;
      grid-template-columns: clamp(110px, 7.5vw, 150px) 1fr clamp(110px, 7.5vw, 150px);
      gap: 10px;
      padding: 10px;
      box-sizing: border-box;
    }

    .panel {
      position: relative;
      z-index: 5;
      height: 100%;
      border: 1px solid rgba(0,0,0,0.12);
      border-radius: 16px;
      background: #fff;
      box-shadow: 0 8px 20px rgba(0,0,0,0.09);
      padding: 10px;
      box-sizing: border-box;
      display: flex;
      flex-direction: column;
      gap: 8px;
      min-width: 0;
    }

    .panelKicker {
      font-size: 10px;
      letter-spacing: 0.14em;
      text-transform: uppercase;
      opacity: 0.70;
      font-weight: 800;
    }

    .panelSliderWrap{
      flex: 1;
      min-height: 0;
      display: flex;
      align-items: center;
      justify-content: center;
    }

    .sliderStack{
      position: relative;
      height: 100%;
      min-height: 0;
      display: flex;
      align-items: stretch;
      gap: 8px;
      padding: 4px 0;
      box-sizing: border-box;
    }

    /* Slim ticks column */
    .vTicks{
      position: relative;
      width: 14px;
      height: 100%;
      opacity: 0.9;
    }

    .tickDot{
      position: absolute;
      left: 50%;
      width: 6px;
      height: 6px;
      border-radius: 999px;
      background: rgba(0,0,0,0.16);
      transform: translate(-50%, -50%);
      transition: transform 120ms ease, background 120ms ease, box-shadow 120ms ease;
    }

    .tickDot.isActive{
      background: rgba(124,58,237,0.95);
      transform: translate(-50%, -50%) scale(1.18);
      box-shadow: 0 0 0 3px rgba(124,58,237,0.14);
    }

    /* Slim slider */
    .vSlider{
      writing-mode: bt-lr;
      width: 22px;
      height: 100%;
      padding: 0;
      margin: 0;
      background: transparent;
    }

    .vSlider.pretty{
      -webkit-appearance: slider-vertical;
      appearance: slider-vertical;
    }

    .vSlider.pretty::-webkit-slider-runnable-track{
      width: 12px;
      background: rgba(0,0,0,0.10);
      border-radius: 999px;
      border: 1px solid rgba(0,0,0,0.10);
      box-shadow: inset 0 0 0 1px rgba(255,255,255,0.55);
    }

    .vSlider.pretty::-webkit-slider-thumb{
      -webkit-appearance: none;
      width: 20px;
      height: 20px;
      border-radius: 999px;
      background: rgba(124,58,237,0.95);
      border: 2px solid #fff;
      box-shadow: 0 6px 14px rgba(0,0,0,0.16);
      margin-top: -4px;
    }

    .vSlider.pretty::-moz-range-track{
      width: 12px;
      background: rgba(0,0,0,0.10);
      border-radius: 999px;
      border: 1px solid rgba(0,0,0,0.10);
    }

    .vSlider.pretty::-moz-range-thumb{
      width: 20px;
      height: 20px;
      border-radius: 999px;
      background: rgba(124,58,237,0.95);
      border: 2px solid #fff;
      box-shadow: 0 6px 14px rgba(0,0,0,0.16);
    }

    /* Smaller floating word label */
    .thumbLabel{
      z-index: 5000;
      position: absolute;
      left: calc(100% + 8px);
      top: 50%;
      transform: translateY(-50%);
      font-size: 11px;
      font-weight: 900;
      padding: 5px 9px;
      border-radius: 999px;
      border: 1px solid rgba(0,0,0,0.12);
      background: rgba(250,250,250,0.96);
      white-space: nowrap;
      pointer-events: none;
      box-shadow: 0 6px 14px rgba(0,0,0,0.10);
    }

    #rightPanel .thumbLabel{
      left: auto;
      right: calc(100% + 8px);
    }

    .mapPanel {
      position: relative;
      z-index: 1;
      border: 1px solid rgba(0,0,0,0.12);
      border-radius: 18px;
      overflow: hidden;
      box-shadow: 0 10px 26px rgba(0,0,0,0.10);
      background: #f3f3f3;
      min-height: 0;
    }

    #map { width: 100%; height: 100%; }

    .leaflet-control-attribution { display: none !important; }
    .leaflet-top, .leaflet-bottom { z-index: 1000; }

    .legend {
      background: rgba(255,255,255,0.92);
      padding: 8px 10px;
      border-radius: 10px;
      box-shadow: 0 1px 10px rgba(0,0,0,0.12);
      line-height: 1.2;
      color: #111;
      font-size: 12px;
      font-family: Inter, Arial, sans-serif;
    }
    .legend .title { font-weight: 700; margin-bottom: 7px; }

    /* align icons + text on one line */
    .legend .item { display:flex; align-items:center; gap:7px; margin: 5px 0; }

    /* squares in legend should not be rounded */
    .legend .swatch {
      width: 12px;
      height: 12px;
      border-radius: 0px;
      border: 1px solid rgba(0,0,0,0.25);
      flex: 0 0 auto;
      margin: 0;
    }

    .legend .note { margin-top: 7px; color: #333; }
    .legend .muted { color:#444; }

    /* filled business center circle in legend */
    .legend .dot {
      width: 10px;
      height: 10px;
      border-radius: 50%;
      border: 2px solid #111;
      background: #f0805a;
      flex: 0 0 auto;
      margin: 0;
    }

    /* critical: constrain legend icons */
    .legend img.icon {
      width: 22px;
      height: 22px;
      object-fit: contain;
      display: block;
      flex: 0 0 auto;
    }


    .barLeft { display: flex; align-items: center; gap: 10px; }
    .barLabel { font-size: 11px; letter-spacing: 0.10em; text-transform: uppercase; opacity: 0.7; white-space: nowrap; }

    .barBtn {
      height: 30px;
      padding: 0 10px;
      border-radius: 999px;
      border: 1px solid rgba(0,0,0,0.18);
      background: rgba(255,255,255,0.92);
      font-size: 12px;
      font-weight: 800;
      cursor: pointer;
      font-family: Inter, Arial, sans-serif;
    }

    .barValue {
      font-size: 13px;
      font-weight: 800;
      white-space: nowrap;
      font-family: Inter, Arial, sans-serif;
      opacity: 0.92;
    }

    @media (max-width: 560px) {
      #topRow {
        grid-template-columns: 1fr;
        grid-template-rows: auto minmax(240px, 1fr) auto;
      }
      .mapPanel { min-height: 45vh; }

      .vSlider { height: 160px; }
      .vTicks { height: 160px; }

      #rightPanel .thumbLabel{ right: calc(100% + 8px); }
      .thumbLabel{ left: calc(100% + 8px); }
    }
  </style>
"""
    if not style_pattern.search(html_str):
        raise ValueError("Could not find a <style> block to replace.")
    html_str = style_pattern.sub(new_style, html_str, count=1)

    # Patch tau meaning if present
    tau_pattern = re.compile(
        r"const\s+tau\s*=\s*tFactor\s*\*\s*\(\s*d\s*/\s*speed_m_per_min\s*\)\s*;",
        re.DOTALL
    )
    if tau_pattern.search(html_str):
        html_str = tau_pattern.sub(
            "const tau = (d / speed_m_per_min) / Math.max(tFactor, 1e-9);",
            html_str,
            count=1
        )

    # After tile add, invalidate size (optional)
    tile_add_pattern = re.compile(r"\}\)\.addTo\(map\);\s*", re.DOTALL)
    if tile_add_pattern.search(html_str):
        html_str = tile_add_pattern.sub(
            "}).addTo(map);\n\n    setTimeout(() => map.invalidateSize(), 0);\n\n    ",
            html_str,
            count=1
        )

    # Update muInfo text to a compact summary (uses sep + areaKm2 in your JS)
    muinfo_el_pattern = re.compile(
        r"muInfoEl\.textContent\s*=\s*`.*?`;\s*",
        re.DOTALL
    )
    if muinfo_el_pattern.search(html_str):
        html_str = muinfo_el_pattern.sub(
            "muInfoEl.textContent = `Distance from Historic to Business Center: ${sep.toFixed(0)}m; Business area: ${areaKm2.toFixed(0)} km²`;\n",
            html_str,
            count=1
        )
    else:
        muinfo_doc_pattern = re.compile(
            r"document\.getElementById\('muInfo'\)\.textContent\s*=\s*`.*?`;\s*",
            re.DOTALL
        )
        if muinfo_doc_pattern.search(html_str):
            html_str = muinfo_doc_pattern.sub(
                "document.getElementById('muInfo').textContent = `Distance from Historic to Business Center: ${sep.toFixed(0)}m; Business area: ${areaKm2.toFixed(0)} km²`;\n",
                html_str,
                count=1
            )

    # Center button wiring (if expected map init exists)
    map_init_hook = re.compile(
        r"const\s+map\s*=\s*L\.map\(\s*(['\"])map\1(?:\s*,[^)]*)?\)\s*"
        r"\.setView\(\s*\[\s*g\.lat\s*,\s*g\.lon\s*\]\s*,\s*11\s*\)\s*;\s*",
        re.DOTALL
    )

    def _inject_center(m: re.Match) -> str:
        original = m.group(0)
        add = (
            "\n    const centerBtn = document.getElementById('centerBtn');\n"
            "    if (centerBtn) {\n"
            "      centerBtn.addEventListener('click', () => {\n"
            "        map.setView([g.lat, g.lon], 11, { animate: true });\n"
            "      });\n"
            "    }\n"
        )
        return original + add

    if map_init_hook.search(html_str):
        html_str = map_init_hook.sub(_inject_center, html_str, count=1)

    # Inject slider snapping + words (unchanged)
    injected_block = r"""
    // ---- injected: symmetric word sliders (0..2, baseline=1) ----
    const T_MIN = 0.0, T_MAX = 2.0;
    const A_MIN = 0.0, A_MAX = 2.0;

    const TRANSPORT_STOPS = [0.50, 0.75, 1.00, 1.25, 1.50];
    const TRANSPORT_WORDS = ["Very slow", "Slower", "Baseline", "Faster", "Very fast"];

    const AMENITY_STOPS = [0.50, 0.75, 1.00, 1.25, 1.50];
    const AMENITY_WORDS = ["Very weak", "Weaker", "Baseline", "Stronger", "Very strong"];

    function clamp(x, lo, hi){ return Math.max(lo, Math.min(hi, x)); }

    function valueToTopPct(v, vmin, vmax) {
      const p = (clamp(v, vmin, vmax) - vmin) / (vmax - vmin);
      return (100 * (1 - p));
    }

    function nearestStop(v, stops) {
      let bestI = 0;
      let bestD = Infinity;
      for (let i = 0; i < stops.length; i++) {
        const d = Math.abs(v - stops[i]);
        if (d < bestD) { bestD = d; bestI = i; }
      }
      return { val: stops[bestI], idx: bestI };
    }

    function buildTicks(elId, stops, vmin, vmax) {
      const el = document.getElementById(elId);
      if (!el) return [];
      el.innerHTML = "";
      const dots = [];
      for (let i = 0; i < stops.length; i++) {
        const d = document.createElement("div");
        d.className = "tickDot";
        d.style.top = valueToTopPct(stops[i], vmin, vmax).toFixed(2) + "%";
        el.appendChild(d);
        dots.push(d);
      }
      return dots;
    }

    const _tDots = buildTicks("tTicks", TRANSPORT_STOPS, T_MIN, T_MAX);
    const _aDots = buildTicks("aTicks", AMENITY_STOPS, A_MIN, A_MAX);

    function setActiveDot(dots, idx){
      for (let i = 0; i < dots.length; i++) dots[i].classList.toggle("isActive", i === idx);
    }

    function positionLabel(labelEl, v, vmin, vmax) {
      if (!labelEl) return;
      labelEl.style.top = valueToTopPct(v, vmin, vmax).toFixed(2) + "%";
    }

    function snapSlider(sliderEl, stops) {
      if (!sliderEl) return { idx: 0, val: NaN };
      const v = parseFloat(sliderEl.value);
      const ns = nearestStop(v, stops);
      sliderEl.value = String(ns.val.toFixed(2));
      return ns;
    }

    function updateWordsAndTicks() {
      const tSlider = document.getElementById("tSlider");
      const aSlider = document.getElementById("aSlider");
      const tLabel = document.getElementById("tVal");
      const aLabel = document.getElementById("aVal");

      if (!tSlider || !aSlider) return;

      const tV = parseFloat(tSlider.value);
      const aV = parseFloat(aSlider.value);

      const tN = nearestStop(tV, TRANSPORT_STOPS);
      const aN = nearestStop(aV, AMENITY_STOPS);

      if (tLabel) tLabel.textContent = TRANSPORT_WORDS[tN.idx];
      if (aLabel) aLabel.textContent = AMENITY_WORDS[aN.idx];

      positionLabel(tLabel, tV, T_MIN, T_MAX);
      positionLabel(aLabel, aV, A_MIN, A_MAX);

      setActiveDot(_tDots, tN.idx);
      setActiveDot(_aDots, aN.idx);
    }

    if (typeof update === "function") {
      const _origUpdate = update;
      update = function() {
        const tSlider = document.getElementById("tSlider");
        const aSlider = document.getElementById("aSlider");
        snapSlider(tSlider, TRANSPORT_STOPS);
        snapSlider(aSlider, AMENITY_STOPS);
        _origUpdate();
        updateWordsAndTicks();
      };
    }

    const _t = document.getElementById("tSlider");
    const _a = document.getElementById("aSlider");
    if (_t) _t.addEventListener("input", () => { snapSlider(_t, TRANSPORT_STOPS); updateWordsAndTicks(); });
    if (_a) _a.addEventListener("input", () => { snapSlider(_a, AMENITY_STOPS); updateWordsAndTicks(); });

    setTimeout(updateWordsAndTicks, 0);
    // ---- end injected block ----
"""

    insert_points = [
        "document.getElementById('aSlider').addEventListener('input', scheduleUpdate);",
        "document.getElementById('tSlider').addEventListener('input', scheduleUpdate);",
        "// Initial draw",
    ]

    inserted = False
    for needle in insert_points:
        if needle in html_str:
            html_str = html_str.replace(needle, needle + "\n" + injected_block, 1)
            inserted = True
            break

    if not inserted:
        idx = html_str.rfind("</script>")
        if idx == -1:
            raise ValueError("Could not find </script> to inject slider logic.")
        html_str = html_str[:idx] + "\n" + injected_block + "\n" + html_str[idx:]

    return html_str



# =========================
# Landing (index.html) generator helpers
# =========================
def build_steps_html(steps: list[dict]) -> str:
    out = []
    for st in steps:
        title = _escape(st.get("title", ""))
        heading = _escape(st.get("heading", st.get("title", "Step")))
        body = _escape(st.get("body", ""))
        hint = _escape(st.get("hint", ""))

        t = float(st.get("t", 1.0))
        a = float(st.get("a", 1.0))

        view_lat = st.get("view_lat", "")
        view_lon = st.get("view_lon", "")
        view_zoom = st.get("view_zoom", "")

        prog = st.get("prog", "")
        idle = bool(st.get("idle", False))
        pad_vh = st.get("pad_vh", None)

        hint_html = f"<p class='stepHint'>{hint}</p>" if hint else ""

        step_classes = "step" + (" stepIdle" if idle else "")
        style_attr = ""
        if pad_vh is not None:
            try:
                pv = float(pad_vh)
                style_attr = f' style="--stepPad:{pv:.1f}vh;"'
            except Exception:
                style_attr = ""

        prog_attr = ""
        try:
            prog_f = float(prog)
            if prog_f == prog_f:
                prog_attr = f' data-prog="{prog_f:.2f}"'
        except Exception:
            prog_attr = ""

        if idle:
            card_html = """<div class="stepCard stepCardIdle" aria-hidden="true"></div>"""
        else:
            card_html = f"""
              <div class="stepCard">
                <div class="stepKicker">Scenario</div>
                <h3>{heading}</h3>
                <p class="stepBody">{body}</p>
                {hint_html}
                <div class="chipRow">
                  <span class="chip">Transport: {t:.2f}</span>
                  <span class="chip">Amenity: {a:.2f}</span>
                </div>
              </div>
            """

        out.append(
            f"""
            <section class="{step_classes}"{style_attr}
              data-title="{title}"
              data-t="{t:.2f}"
              data-a="{a:.2f}"
              data-view-lat="{view_lat}"
              data-view-lon="{view_lon}"
              data-view-zoom="{view_zoom}"{prog_attr}>
              {card_html}
            </section>
            """
        )
    return "\n".join(out)


def build_gallery_html(items, cols=3):
    cards = []
    for it in items:
        src = _escape(it["src"])
        cap = _escape(it.get("caption", ""))
        cap_html = f"<figcaption>{cap}</figcaption>" if cap else ""
        cards.append(
            f"""
            <figure class="imgCard">
              <img src="{src}" alt="{cap}" loading="lazy"/>
              {cap_html}
            </figure>
            """
        )
    return f"""<div class="imgGrid cols{int(cols)}">{''.join(cards)}</div>"""


def build_explain_blocks(blocks):
    out = []
    for b in blocks:
        h = _escape(b.get("heading", ""))
        p = _escape(b.get("body", ""))
        out.append(
            f"""
            <div class="textCard">
              <h3>{h}</h3>
              <p>{p}</p>
            </div>
            """
        )
    return "\n".join(out)


# =========================
# Dashboard page generator (fixed: removed unused height_px)
# =========================
def build_dashboard_page_html(
    page_title: str,
    iframe_src: str,
    iframe_title: str = "Theoretical modelling dashboard",
) -> str:
    return f"""<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <title>{_escape(page_title)}</title>

  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">

  <style>
    :root{{
      --bg: #fafafa;
      --panel: #ffffff;
      --text: #0f0f10;
      --muted: rgba(15,15,16,.68);
      --line: rgba(15,15,16,.12);
      --radius2: 24px;
      --padX: 22px;
      --navH: 68px; /* set by JS */
    }}

    html, body {{ height: 100%; }}
    body{{
      margin:0;
      font-family: Inter, Arial, sans-serif;
      background: var(--bg);
      color: var(--text);
      display: flex;
      flex-direction: column;
    }}
    a{{ color: inherit; text-decoration: none; }}

    .nav{{
      position: sticky;
      top: 0;
      z-index: 50;
      background: rgba(250,250,250,.86);
      backdrop-filter: blur(10px);
      border-bottom: 1px solid var(--line);
    }}

    /* full width */
    .container{{
      width: 100%;
      box-sizing: border-box;
      padding: 0 var(--padX);
    }}

    .navInner{{
      display:flex;
      align-items:center;
      justify-content:space-between;
      padding: 14px 0;
      gap: 16px;
    }}
    .brand{{ font-weight: 700; letter-spacing: -0.02em; }}

    .btn{{
      display:inline-flex;
      align-items:center;
      justify-content:center;
      height: 40px;
      padding: 0 14px;
      border-radius: 999px;
      border: 1px solid var(--line);
      background: rgba(255,255,255,.86);
      font-weight: 600;
      font-size: 13px;
      cursor: pointer;
    }}

    /* main fills remaining viewport height */
    main{{
      flex: 1;
      display: flex;
      flex-direction: column;
      padding: 18px 0 22px 0;
      min-height: 0;
    }}

    .meta{{
      flex: 0 0 auto;
      margin: 0 0 12px 0;
      color: var(--muted);
      font-size: 13px;
      line-height: 1.5;
    }}

    .card{{
      flex: 1 1 auto;
      min-height: 0;
      background: var(--panel);
      border: 1px solid var(--line);
      border-radius: var(--radius2);
      overflow: hidden;
    }}

    iframe{{
      width: 100%;
      height: 100%;
      border: 0;
      display: block;
      background: #fff;
    }}
  </style>
</head>

<body>
  <div class="nav" id="navEl">
    <div class="container">
      <div class="navInner">
        <div class="brand">{_escape(page_title)}</div>
        <a class="btn" href="index.html#explain">Back</a>
      </div>
    </div>
  </div>

  <main class="container">
    <div class="meta" id="metaEl">{_escape(iframe_title)}</div>
    <div class="card" id="cardEl">
      <iframe id="dashFrame" src="{_escape(iframe_src)}" title="{_escape(iframe_title)}" loading="eager"></iframe>
    </div>
  </main>

  <script>
    (function() {{
      const nav = document.getElementById("navEl");
      const meta = document.getElementById("metaEl");
      const card = document.getElementById("cardEl");

      function sync() {{
        const navH = nav ? (nav.getBoundingClientRect().height || 68) : 68;
        document.documentElement.style.setProperty("--navH", Math.round(navH) + "px");

        /* Ensure the card is always visible and not taller than the viewport */
        const vh = window.innerHeight || 800;
        const metaH = meta ? (meta.getBoundingClientRect().height || 0) : 0;

        /* main has padding top/bottom (18 + 22) = 40 */
        const pad = 40;
        const target = Math.max(520, Math.floor(vh - navH - metaH - pad));
        if (card) card.style.height = target + "px";
      }}

      window.addEventListener("resize", sync);
      setTimeout(sync, 0);
      setTimeout(sync, 250);
    }})();
  </script>
</body>
</html>
"""


# =========================
# Landing (index.html) generator
# =========================
def build_landing_html(config: dict) -> str:
    body_class_attr = ' class="modelLayoutFocus"'

    t = Template(r"""<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <title>${PAGE_TITLE}</title>

  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">

  <style>
    :root{
      --bg: #fafafa;
      --panel: #ffffff;
      --text: #0f0f10;
      --muted: rgba(15,15,16,.68);
      --line: rgba(15,15,16,.12);
      --shadow: 0 14px 40px rgba(0,0,0,.08);
      --radius2: 24px;
      --max: 1160px;
    }

    html{ scroll-behavior: smooth; }
    body{
      margin:0;
      font-family: Inter, Arial, sans-serif;
      background: var(--bg);
      color: var(--text);
    }
    a{ color: inherit; text-decoration: none; }
    .container{ max-width: var(--max); margin: 0 auto; padding: 0 22px; }

    .nav{
      position: sticky;
      top: 0;
      z-index: 50;
      background: rgba(250,250,250,.86);
      backdrop-filter: blur(10px);
      border-bottom: 1px solid var(--line);
    }
    .navInner{
      display:flex;
      align-items:center;
      justify-content:space-between;
      padding: 14px 0;
      gap: 16px;
    }
    .brand{ font-weight: 700; letter-spacing: -0.02em; }
    .navLinks{ display:flex; gap: 14px; flex-wrap: wrap; justify-content: flex-end; }
    .navLinks a{
      font-size: 13px;
      color: var(--muted);
      padding: 8px 10px;
      border-radius: 999px;
      border: 1px solid transparent;
    }
    .navLinks a:hover{
      color: var(--text);
      border-color: var(--line);
      background: rgba(255,255,255,.7);
    }

    .hero{
      padding: 62px 0 36px 0;
      border-bottom: 1px solid var(--line);
    }
    .heroGrid{
      display:grid;
      grid-template-columns: 1.1fr .9fr;
      gap: 22px;
      align-items: start;
    }
    .kicker{
      font-size: 12px;
      color: var(--muted);
      letter-spacing: .12em;
      text-transform: uppercase;
      margin-bottom: 14px;
    }
    h1{
      font-size: clamp(34px, 4.2vw, 60px);
      line-height: 1.02;
      letter-spacing: -0.04em;
      margin: 0 0 14px 0;
    }
    .sub{
      font-size: 16px;
      color: var(--muted);
      line-height: 1.6;
      max-width: 54ch;
      margin: 0 0 22px 0;
    }
    .ctaRow{ display:flex; gap: 10px; flex-wrap: wrap; align-items:center; }
    .btn{
      display:inline-flex;
      align-items:center;
      justify-content:center;
      height: 44px;
      padding: 0 16px;
      border-radius: 999px;
      border: 1px solid var(--line);
      background: rgba(255,255,255,.86);
      font-weight: 600;
      font-size: 13px;
    }
    .btnPrimary{
      background: #111;
      color: #fff;
      border-color: #111;
    }

    .heroCard{
      background: var(--panel);
      border: 1px solid var(--line);
      border-radius: var(--radius2);
      overflow:hidden;
      box-shadow: 0 1px 0 rgba(0,0,0,.04);
    }
    .heroCard img{ width:100%; height: 360px; object-fit: cover; display:block; }
    .heroMeta{ padding: 14px 14px 16px 14px; color: var(--muted); font-size: 13px; line-height: 1.5; }

    .section{ padding: 46px 0; }
    .sectionTitle{
      display:flex;
      justify-content: space-between;
      align-items: baseline;
      gap: 12px;
      margin-bottom: 18px;
    }
    .sectionTitle h2{ margin:0; font-size: 22px; letter-spacing: -0.02em; }
    .sectionTitle p{ margin:0; color: var(--muted); font-size: 13px; max-width: 58ch; line-height: 1.5; }

    .imgGrid{ display:grid; gap: 14px; }
    .imgGrid.cols3{ grid-template-columns: repeat(3, 1fr); }
    .imgGrid.cols2{ grid-template-columns: repeat(2, 1fr); }
    .imgCard{
      margin: 0;
      background: var(--panel);
      border: 1px solid var(--line);
      border-radius: var(--radius2);
      overflow:hidden;
      box-shadow: 0 1px 0 rgba(0,0,0,.04);
    }
    .imgCard img{ width: 100%; height: 280px; object-fit: cover; display:block; }
    .imgCard figcaption{ padding: 12px 14px; font-size: 13px; color: var(--muted); line-height: 1.4; }

    .modelWrap{
      border-top: 1px solid var(--line);
      border-bottom: 1px solid var(--line);
      background: #fff;
    }
    .modelHeader{ padding: 46px 0 14px 0; }
    .modelGrid{
      display:grid;
      grid-template-columns: 1fr 1.2fr;
      gap: 18px;
      padding-bottom: 48px;
    }
    .stepsCol{ padding-right: 10px; }

    .stickyViz{
      position: sticky;
      top: 74px;
      height: calc(100vh - 96px);
      border: 1px solid var(--line);
      border-radius: var(--radius2);
      overflow:hidden;
      background: #f3f3f3;
      box-shadow: var(--shadow);
    }
    .stickyViz iframe{ width: 100%; height: 100%; border: 0; display:block; background: #fff; }

    .modelMiniBar{
      display:flex;
      align-items:center;
      justify-content:space-between;
      gap: 14px;
      padding: 10px 0 16px 0;
      color: var(--muted);
      font-size: 13px;
    }
    .progress{
      height: 6px;
      flex: 1;
      background: rgba(0,0,0,.08);
      border-radius: 999px;
      overflow:hidden;
    }
    .progress > div{ height: 100%; width: 0%; background: #111; }

    /* Step padding is now adjustable via CSS variable per step */
    .step{ padding: var(--stepPad, 24vh) 0; }
    .step:first-child:not(.stepIdle){ padding-top: 14vh; }
    .step:last-child:not(.stepIdle){ padding-bottom: 14vh; }

    .stepCard{
      background: #fff;
      border: 1px solid var(--line);
      border-radius: var(--radius2);
      padding: 16px 16px 14px 16px;
      box-shadow: 0 1px 0 rgba(0,0,0,.04);
      transition: transform .15s ease, box-shadow .15s ease, border-color .15s ease;
    }
    .step.is-active .stepCard{
      border-color: rgba(0,0,0,.55);
      box-shadow: 0 18px 46px rgba(0,0,0,.10);
      transform: translateY(-1px);
    }
    .stepKicker{ font-size: 11px; letter-spacing: .12em; text-transform: uppercase; color: var(--muted); margin-bottom: 8px; }
    .stepCard h3{ margin: 0 0 8px 0; font-size: 18px; letter-spacing: -0.02em; }
    .stepBody{ margin: 0 0 10px 0; color: rgba(0,0,0,.72); font-size: 14px; line-height: 1.55; }
    .stepHint{ margin: 0 0 10px 0; color: var(--muted); font-size: 13px; line-height: 1.5; }
    .chipRow{ display:flex; gap: 8px; flex-wrap: wrap; margin-top: 6px; }
    .chip{ display:inline-flex; padding: 6px 10px; border-radius: 999px; border: 1px solid var(--line); background: rgba(250,250,250,.9); font-size: 12px; color: rgba(0,0,0,.72); }

    /* Idle spacer steps: keep scroll height but hide the card */
    .stepIdle .stepCard{
      opacity: 0;
      pointer-events: none;
      border-color: transparent;
      box-shadow: none;
      transform: none;
    }
    .stepIdle.is-active .stepCard{
      border-color: transparent;
      box-shadow: none;
      transform: none;
    }

    .explainGrid{ display:grid; grid-template-columns: 1fr 1fr; gap: 14px; margin-top: 14px; }
    .textCard{
      background: var(--panel);
      border: 1px solid var(--line);
      border-radius: var(--radius2);
      padding: 18px 18px 16px 18px;
      box-shadow: 0 1px 0 rgba(0,0,0,.04);
    }
    .textCard h3{ margin: 0 0 8px 0; font-size: 16px; }
    .textCard p{ margin:0; color: rgba(0,0,0,.72); font-size: 14px; line-height: 1.6; }

    footer{ border-top: 1px solid var(--line); padding: 26px 0 40px 0; color: var(--muted); font-size: 13px; line-height: 1.5; }

    @media (max-width: 980px){
      .heroGrid{ grid-template-columns: 1fr; }
      .modelGrid{ grid-template-columns: 1fr; }
      .stickyViz{ position: relative; top: 0; height: 70vh; }
      .imgGrid.cols3{ grid-template-columns: 1fr; }
      .imgGrid.cols2{ grid-template-columns: 1fr; }
      .explainGrid{ grid-template-columns: 1fr; }
      .step{ padding: var(--stepPad, 16vh) 0; }
    }
  </style>
</head>

<body id="top""" + body_class_attr + r""">
  <div class="nav">
    <div class="container">
      <div class="navInner">
        <div class="brand">${BRAND}</div>
        <div class="navLinks">
          <a href="#gallery1">Gallery</a>
          <a href="#model">Model</a>
          <a href="#gallery2">More</a>
          <a href="#explain">Explanation</a>
        </div>
      </div>
    </div>
  </div>

  <header class="hero">
    <div class="container">
      <div class="heroGrid">
        <div>
          <div class="kicker">${HERO_KICKER}</div>
          <h1>${HERO_TITLE}</h1>
          <p class="sub">${HERO_SUB}</p>
          <div class="ctaRow">
            <a class="btn btnPrimary" href="#model">${CTA_PRIMARY}</a>
            <a class="btn" href="#explain">${CTA_SECONDARY}</a>
          </div>
        </div>

        <div class="heroCard">
          <img src="${HERO_IMAGE}" alt="" loading="eager">
          <div class="heroMeta">${HERO_IMAGE_CAPTION}</div>
        </div>
      </div>
    </div>
  </header>

  <section class="section" id="gallery1">
    <div class="container">
      <div class="sectionTitle">
        <h2>${G1_TITLE}</h2>
        <p>${G1_SUB}</p>
      </div>
      ${GALLERY1_HTML}
    </div>
  </section>

  <section class="modelWrap" id="model">
    <div class="container">
      <div class="modelHeader">
        <div class="sectionTitle">
          <h2>${MODEL_TITLE}</h2>
          <p>${MODEL_SUB}</p>
        </div>

        <div class="modelMiniBar">
          <div id="stepName">Baseline</div>
          <div class="progress"><div id="progBar"></div></div>
        </div>
      </div>

      <div class="modelGrid">
        <div class="stepsCol">
          ${STEPS_HTML}
        </div>

        <div class="stickyViz">
          <iframe id="viz" src="${VIZ_FILENAME}" title="Interactive model" loading="lazy"></iframe>
        </div>
      </div>
    </div>
  </section>

  <section class="section" id="gallery2">
    <div class="container">
      <div class="sectionTitle">
        <h2>${G2_TITLE}</h2>
        <p>${G2_SUB}</p>
      </div>
      ${GALLERY2_HTML}
    </div>
  </section>

  <section class="section" id="explain">
    <div class="container">
      <div class="sectionTitle">
        <h2>${EXPLAIN_TITLE}</h2>
        <p>${EXPLAIN_SUB}</p>
      </div>
      <div class="explainGrid">
        ${EXPLAIN_BLOCKS}
      </div>
      ${EXTRA_EXPLAIN_HTML}
    </div>
  </section>

  <footer>
    <div class="container">
      ${FOOTER_TEXT}
    </div>
  </footer>

  <script src="https://unpkg.com/scrollama"></script>
  <script>
    const iframe = document.getElementById("viz");
    const steps = Array.from(document.querySelectorAll(".step"));
    const stepName = document.getElementById("stepName");
    const progBar = document.getElementById("progBar");
    const stickyVizEl = document.querySelector("#model .stickyViz");
    const navEl = document.querySelector(".nav");

    const pending = { sliders: null, view: null, invalidate: false };

    function _postToViz(msg) {
      if (!iframe || !iframe.contentWindow) return;
      iframe.contentWindow.postMessage(msg, "*");
    }

    function _remember(msg) {
      if (!msg || typeof msg !== "object") return;
      if (msg.type === "setSliders") pending.sliders = msg;
      if (msg.type === "setView") pending.view = msg;
      if (msg.type === "invalidateSize") pending.invalidate = true;
    }

    function replayPending() {
      if (!iframe || !iframe.contentWindow) return;
      _postToViz({ type: "ping" });
      if (pending.sliders) _postToViz(pending.sliders);
      if (pending.view) _postToViz(pending.view);
      if (pending.invalidate) _postToViz({ type: "invalidateSize" });
    }

    /* Reliable sendToViz (already safe, no extra patch needed) */
    function sendToViz(msg) {
      _remember(msg);
      if (!iframe || !iframe.contentWindow) return;
      _postToViz(msg);
    }

    if (iframe) {
      iframe.addEventListener("load", () => {
        replayPending();
        sendToViz({ type: "invalidateSize" });
      });
    }

    function syncNavHeightVar() {
      if (!navEl) return;
      const h = navEl.getBoundingClientRect().height || 68;
      document.documentElement.style.setProperty("--navH", Math.round(h) + "px");
    }

    window.addEventListener("resize", () => {
      syncNavHeightVar();
      schedulePinnedCheck();
    });

    setTimeout(syncNavHeightVar, 0);

    let lastIdx = -1;
    function activateStep(el, idx) {
      if (!el) return;
      if (idx === lastIdx) return;
      lastIdx = idx;

      steps.forEach(s => s.classList.remove("is-active"));
      el.classList.add("is-active");

      const title = el.dataset.title || ("Step " + String(idx + 1));
      if (stepName) stepName.textContent = title;

      // Skip iframe updates for idle spacer steps
      if (el.classList.contains("stepIdle")) {
        const pctIdle = parseFloat(el.dataset.prog);
        if (progBar && isFinite(pctIdle)) {
          progBar.style.width = String(pctIdle.toFixed(1)) + "%";
        }
        return;
      }

      const t = parseFloat(el.dataset.t);
      const a = parseFloat(el.dataset.a);

      const lat = parseFloat(el.dataset.viewLat);
      const lon = parseFloat(el.dataset.viewLon);
      const zoom = parseFloat(el.dataset.viewZoom);

      if (isFinite(t) && isFinite(a)) sendToViz({ type: "setSliders", t: t, a: a });
      if (isFinite(lat) && isFinite(lon) && isFinite(zoom)) sendToViz({ type: "setView", lat: lat, lon: lon, zoom: zoom });

      const pct = parseFloat(el.dataset.prog);
      if (progBar) {
        if (isFinite(pct)) {
          progBar.style.width = String(pct.toFixed(1)) + "%";
        } else {
          const fallback = steps.length <= 1 ? 100 : (idx / (steps.length - 1)) * 100;
          progBar.style.width = String(fallback.toFixed(1)) + "%";
        }
      }
    }

    // Minibar: show only when sticky map is pinned
    let pinnedOn = false;

    function setPinned(on) {
      const next = !!on;
      if (pinnedOn === next) return;
      pinnedOn = next;
      document.body.classList.toggle("modelPinned", pinnedOn);

      setTimeout(() => {
        sendToViz({ type: "invalidateSize" });
        replayPending();
      }, 140);
    }

    function computePinned() {
      if (!stickyVizEl) { setPinned(false); return; }

      const cs = getComputedStyle(stickyVizEl);
      const pos = cs.position;
      if (pos !== "sticky" && pos !== "-webkit-sticky") { setPinned(false); return; }

      const r = stickyVizEl.getBoundingClientRect();
      const topPx = parseFloat(cs.top) || 0;
      const vh = window.innerHeight || 800;

      const delta = Math.abs(r.top - topPx);
      const pinned = pinnedOn ? (delta <= 26) : (delta <= 2);
      const visible = r.bottom > (topPx + 140) && r.top < (vh - 140);

      setPinned(pinned && visible);
    }

    let pinRAF = 0;
    function schedulePinnedCheck() {
      cancelAnimationFrame(pinRAF);
      pinRAF = requestAnimationFrame(computePinned);
    }

    window.addEventListener("scroll", schedulePinnedCheck, { passive: true });

    if (stickyVizEl && "ResizeObserver" in window) {
      const ro = new ResizeObserver(() => {
        setTimeout(() => sendToViz({ type: "invalidateSize" }), 140);
      });
      ro.observe(stickyVizEl);
    }

    const scroller = scrollama();
    scroller
      .setup({ step: ".step", offset: 0.62 })
      .onStepEnter((resp) => {
        schedulePinnedCheck();
        activateStep(resp.element, resp.index);
      })
      .onStepExit((resp) => {
        if (resp.direction === "up" && resp.index > 0) {
          const prev = resp.index - 1;
          activateStep(steps[prev], prev);
        }
        schedulePinnedCheck();
      });

    window.addEventListener("message", (event) => {
      if (event.data && event.data.type === "vizReady") {
        replayPending();
        sendToViz({ type: "invalidateSize" });
      }
    });

    // Init
    syncNavHeightVar();
    schedulePinnedCheck();
    setTimeout(() => {
      if (steps.length) activateStep(steps[0], 0);
      replayPending();
      sendToViz({ type: "invalidateSize" });
      schedulePinnedCheck();
    }, 700);
  </script>

</body>
</html>
""")
    return t.substitute(config)


# =========================
# One function to write full site
# =========================
def write_full_scrolly_site(
    out_dir: str,
    interactive_html: str,
    viz_filename: str = "yerevan_continuous_two_sliders.html",
    landing_filename: str = "index.html",
    title: str = "Yerevan scrolly",
    extra_html_files: dict[str, str] | None = None,
    embed_extra_filename: str | None = None,
    compare_html_src_path: str | None = None,
    compare_filename: str = "compare_business_areas.html",
):
    os.makedirs(out_dir, exist_ok=True)

    # Patch the interactive: UI + center zoom + scrolly hooks
    viz_html = interactive_html
    viz_html = patch_interactive_ui_left_right_vertical_sliders(viz_html)
    viz_html = patch_center_map_button_zoom(viz_html, center_zoom=12.0)
    viz_html = patch_interactive_for_scrolly(viz_html)

    viz_path = os.path.join(out_dir, viz_filename)
    with open(viz_path, "w", encoding="utf-8") as f:
        f.write(viz_html)

    if extra_html_files:
        for fname, html_text in extra_html_files.items():
            Path(os.path.join(out_dir, fname)).write_text(html_text, encoding="utf-8")

    # Gallery placeholders
    g1 = [
        {"src": svg_placeholder_data_uri("Context image 1"), "caption": "Replace with assets/ images"},
        {"src": svg_placeholder_data_uri("Context image 2"), "caption": "Replace with assets/ images"},
        {"src": svg_placeholder_data_uri("Context image 3"), "caption": "Replace with assets/ images"},
    ]
    g2 = [
        {"src": svg_placeholder_data_uri("More image 1"), "caption": "Replace with assets/ images"},
        {"src": svg_placeholder_data_uri("More image 2"), "caption": "Replace with assets/ images"},
    ]

    # Core scenarios
    scenario_steps = [
        dict(
            title="Baseline",
            heading="Baseline assumptions",
            body="Reference case for transport and amenity strength.",
            hint="Scroll to switch scenarios. The model updates immediately.",
            t=1.00,
            a=1.00
        ),
        dict(
            title="Faster transport",
            heading="Faster transport",
            body="Higher transport speed multiplier lowers time costs.",
            t=1.50,
            a=1.00
        ),
        dict(
            title="Slower transport",
            heading="Slower transport",
            body="Lower transport speed multiplier raises time costs.",
            t=0.50,
            a=1.00
        ),
        dict(
            title="Historic pull",
            heading="Historic amenities matter more",
            body="Higher amenity multiplier strengthens amenity-related effects.",
            t=1.00,
            a=1.50
        ),
        dict(
            title="Weaker amenities",
            heading="Historic amenities matter less",
            body="Lower amenity multiplier weakens amenity-related effects.",
            t=1.00,
            a=0.50
        ),
    ]

    scenario_steps = assign_progress_percent(scenario_steps)
    steps_all = add_idle_spacer_steps(
        scenario_steps,
        lead_count=IDLE_LEAD_STEPS,
        tail_count=IDLE_TAIL_STEPS,
        lead_pad_vh=IDLE_LEAD_PAD_VH,
        tail_pad_vh=IDLE_TAIL_PAD_VH,
    )

    explain_blocks = [
        dict(
            heading="Idle scroll",
            body="Extra invisible spacer steps before the first and after the last scenario keep the model pinned long enough for the minibar and progress to be visible."
        ),
        dict(
            heading="Width",
            body="The model section uses near-full viewport width with small margins."
        ),
        dict(
            heading="Cards",
            body="Scenario cards are fixed-width and compact so the map is large."
        ),
        dict(
            heading="Menu + bar",
            body="The top menu is fixed; the progress bar appears only when the map is pinned and sits below the menu."
        ),
    ]

    dashboard_page = "dashboard.html"
    dashboard_button_html = ""
    if embed_extra_filename:
        dashboard_button_html = f"""
        <div style="display:flex; flex-direction:column; gap:12px; margin-top: 12px;">
          <a class="btn btnPrimary" href="{_escape(dashboard_page)}">Open theoretical dashboard</a>
        </div>
        """

    config = {
        "PAGE_TITLE": title,
        "BRAND": title,
        "HERO_KICKER": "Urban model explorer",
        "HERO_TITLE": "Moving centers with transport and amenities",
        "HERO_SUB": "Big map in the model section, compact scenario cards, fixed menu, and a pinned-only progress bar below the menu.",
        "CTA_PRIMARY": "Explore the model",
        "CTA_SECONDARY": "Read the explanation",
        "HERO_IMAGE": svg_placeholder_data_uri("Hero image", 1800, 1200),
        "HERO_IMAGE_CAPTION": "Replace with a real image in assets/ and update the path.",
        "G1_TITLE": "Context",
        "G1_SUB": "Supporting visuals before the model.",
        "GALLERY1_HTML": build_gallery_html(g1, cols=3),
        "MODEL_TITLE": "Interactive model",
        "MODEL_SUB": "Scroll-driven scenarios with a sticky frame.",
        "STEPS_HTML": build_steps_html(steps_all),
        "VIZ_FILENAME": viz_filename,
        "G2_TITLE": "More visuals",
        "G2_SUB": "More figures or screenshots after the model.",
        "GALLERY2_HTML": build_gallery_html(g2, cols=2),
        "EXPLAIN_TITLE": "Explanation",
        "EXPLAIN_SUB": "Short blocks explaining what is happening.",
        "EXPLAIN_BLOCKS": build_explain_blocks(explain_blocks),
        "EXTRA_EXPLAIN_HTML": "",
        "FOOTER_TEXT": f"""
          <div style="display:flex; flex-direction:column; gap:12px;">
            <div>Static bundle. Serve the folder as a website to share it.</div>
            {dashboard_button_html}
          </div>
        """,
    }

    # Copy compare-business-areas HTML into the site folder (optional)
    if compare_html_src_path:
        if not os.path.exists(compare_html_src_path):
            raise FileNotFoundError(f"Missing compare HTML: {compare_html_src_path}")
        dst = os.path.join(out_dir, compare_filename)
        shutil.copyfile(compare_html_src_path, dst)

        txt = Path(dst).read_text(encoding="utf-8")
        txt = patch_compare_add_height_postmessage(txt)
        Path(dst).write_text(txt, encoding="utf-8")

    landing_html = build_landing_html(config)
    landing_html = patch_landing_for_model_focus_zoom(landing_html)

    # Compare section insertion (optional)
    if compare_html_src_path:
        landing_html = patch_landing_add_compare_embed_css(landing_html)
        landing_html = patch_landing_insert_compare_before_explain(
            landing_html,
            compare_href=compare_filename,
            title="Заголовок блока",
            sub="Сравнение бизнес-ареалов городов. Фигуры приведены к одной шкале и центрированы по историческому центру (0,0).",
        )
        landing_html = patch_landing_add_compare_autosize_js(landing_html)

    landing_path = os.path.join(out_dir, landing_filename)
    with open(landing_path, "w", encoding="utf-8") as f:
        f.write(landing_html)

    if embed_extra_filename:
        dash_html = build_dashboard_page_html(
            page_title=f"{title} | Dashboard",
            iframe_src=embed_extra_filename,
            iframe_title="Theoretical modelling dashboard",
        )
        Path(os.path.join(out_dir, dashboard_page)).write_text(dash_html, encoding="utf-8")

    return viz_path, landing_path


# =========================
# USAGE
# =========================
# You should already have:
#   html_filled = the generated interactive HTML string
#
thesis_dashboard_filename = "thesis_sectioned_dashboard.html"
COMPARE_HTML_SRC = "data/compare_business_areas/compare_business_areas.html"

viz_path, landing_path = write_full_scrolly_site(
    out_dir=OUT_DIR,
    interactive_html=html_filled,
    viz_filename="yerevan_sliders_single_business_polygon.html",
    landing_filename="index.html",
    title="Yerevan scrolly",
    embed_extra_filename=thesis_dashboard_filename,
    compare_html_src_path=COMPARE_HTML_SRC,
    compare_filename="compare_business_areas.html",
)

print(viz_path, landing_path)


data/yerevan_interactive\yerevan_sliders_single_business_polygon.html data/yerevan_interactive\index.html


In [2]:
import os
import base64
import hashlib
import requests
from pathlib import Path

OWNER = "Nedric53"
REPO = "Armenia"
BRANCH = "main"

# Local folder that contains index.html and yerevan_continuous_two_sliders.html
LOCAL_SITE_DIR = Path("data/yerevan_interactive").resolve()

# Where to put the site inside the repo
REMOTE_DIR = ""  # recommended for GitHub Pages

# GitHub token (better: set as environment variable GITHUB_TOKEN)
TOKEN = os.environ.get("GITHUB_TOKEN", "a")

API = "https://api.github.com"

session = requests.Session()
session.headers.update({
    "Authorization": f"token {TOKEN}",
    "Accept": "application/vnd.github+json",
})

def get_file_sha(path_in_repo: str):
    url = f"{API}/repos/{OWNER}/{REPO}/contents/{path_in_repo}"
    r = session.get(url, params={"ref": BRANCH})
    if r.status_code == 200:
        return r.json().get("sha")
    if r.status_code == 404:
        return None
    raise RuntimeError(f"GitHub API error fetching {path_in_repo}: {r.status_code} {r.text}")

def upload_file(local_path: Path, path_in_repo: str, message: str):
    content_bytes = local_path.read_bytes()
    content_b64 = base64.b64encode(content_bytes).decode("utf-8")

    sha = get_file_sha(path_in_repo)

    url = f"{API}/repos/{OWNER}/{REPO}/contents/{path_in_repo}"
    payload = {
        "message": message,
        "content": content_b64,
        "branch": BRANCH,
    }
    if sha:
        payload["sha"] = sha

    r = session.put(url, json=payload)
    if r.status_code not in (200, 201):
        raise RuntimeError(f"Upload failed for {path_in_repo}: {r.status_code} {r.text}")

def iter_files(root: Path):
    for p in root.rglob("*"):
        if p.is_file():
            yield p

if not LOCAL_SITE_DIR.exists():
    raise FileNotFoundError(f"Local site dir not found: {LOCAL_SITE_DIR}")

for p in iter_files(LOCAL_SITE_DIR):
    rel = p.relative_to(LOCAL_SITE_DIR).as_posix()
    repo_path = rel if not REMOTE_DIR else f"{REMOTE_DIR}/{rel}"
    msg = f"Update site: {repo_path}"
    print("Uploading:", repo_path)
    upload_file(p, repo_path, msg)
print("Done. Your site files are in:", f"{REMOTE_DIR}/")



Uploading: business_area_only.geojson
Uploading: business_area_only.html
Uploading: compare_business_areas.html
Uploading: dashboard.html
Uploading: index.html
Uploading: thesis_sectioned_dashboard.html
Uploading: yerevan_business_area_only_empty.html
Uploading: yerevan_business_only.geojson
Uploading: yerevan_business_only.html
Uploading: yerevan_business_polygon_only_sliders.html
Uploading: yerevan_business_polygon_precomputed.html
Uploading: yerevan_continuous_two_sliders.html
Uploading: yerevan_sliders_single_business_polygon.html
Uploading: .ipynb_checkpoints/index-checkpoint.html
Uploading: .ipynb_checkpoints/yerevan_continuous_two_sliders-checkpoint.html
Uploading: assets/context_image_1.png
Uploading: assets/context_image_2.png
Uploading: assets/context_image_3.png
Uploading: assets/icon_business.png
Uploading: assets/icon_historical.png
Uploading: assets/.ipynb_checkpoints/icon_business-checkpoint.png
Done. Your site files are in: /


In [4]:
from pathlib import Path

NOTEBOOK_PATH = Path("Armenia.ipynb").resolve()

if not NOTEBOOK_PATH.exists():
    raise FileNotFoundError(f"Notebook not found: {NOTEBOOK_PATH}")

# Put it in the repo root. Change to "notebooks/Armenia.ipynb" if you prefer a folder.
notebook_repo_path = "Armenia.ipynb"

print("Uploading notebook:", notebook_repo_path)
upload_file(NOTEBOOK_PATH, notebook_repo_path, "Add/update notebook: Armenia.ipynb")
print("Done uploading notebook.")


Uploading notebook: Armenia.ipynb


RuntimeError: Upload failed for Armenia.ipynb: 409 {"message":"Repository rule violations found\n\nSecret detected in content\n\n","metadata":{"secret_scanning":{"bypass_placeholders":[{"placeholder_id":"39Ki2WRiaZJbnK5ojed9Ze1UC3P","token_type":"GITHUB_TOKEN_V2"}]}},"documentation_url":"https://docs.github.com/rest/repos/contents#create-or-update-file-contents","status":"409"}